## Data for VI --> Yield Model
sudo mount ../../dev/disk/by-label/TOSHIBA\\x20EXT ~/Thesis_Work/2023_VI/Raw_Data/drive

### Libraries

In [1]:
import ee # Google earth engine
import geemap # Allows for interactive mapping with earth engine
import pandas as pd # Manipulate data in the form of spreadsheets
import matplotlib.pyplot as plt # plotting 
from matplotlib.pyplot import imshow # plotting 
import numpy as np # Math library
import multiprocessing # Multiprocessing
import pickle # Convienent way to store variables from RAM to disk
import random # for random number generation
import time
import os
import time

import warnings # Remove some annoying warnings (optional)
warnings.filterwarnings('ignore')

In [2]:
ee.Authenticate()

Enter verification code: 4/1AQlEd8xi6_mvBUaRgQFqncWCMkKl9X3ebPB_42WQbCGnW73T9bxSQFW773Y

Successfully saved authorization token.


In [3]:
ee.Initialize() # Initialize earth engine

### 2. Download District Images of VI for Training VI --> Yield Model (250m Resolution)
Specifically, MOD13Q1 data will be downloaded.

In [34]:
# USA_Counties = ee.FeatureCollection('projects/ee-amanjotbhullar/assets/USA_Counties')
# USA_Counties_list = USA_Counties.toList(USA_Counties.size())

# USA_boundaries = []

# for i in range(0, USA_Counties.size().getInfo()):
#     geometry = ee.Feature(USA_Counties_list.get(i))
#     name = geometry.getInfo()['properties']['STATE_FIPS'] + '_' + geometry.getInfo()['properties']['CNTY_FIPS']
#     print(name)
#     USA_boundaries.append((geometry, name))
    
    
# with open('./usa_county_polygons.pickle', 'wb') as file:
#     pickle.dump(USA_boundaries, file)

01_001
01_003
01_005
01_009
01_011
01_015
01_021
01_031
01_045
01_069
01_071
01_073
01_077
01_081
01_083
01_089
01_095
01_097
01_101
01_103
01_113
01_117
01_115
01_125
01_133
01_023
01_027
01_057
01_063
01_085
01_019
01_035
01_039
01_065
01_107
01_131
01_067
01_075
01_087
01_025
01_119
01_105
01_111
01_079
01_121
01_013
01_041
01_061
01_123
01_099
01_017
01_029
01_059
01_047
01_053
01_129
01_007
01_091
01_093
01_037
01_043
01_033
01_109
01_127
02_020
02_090
02_110
02_122
02_063
02_066
04_003
04_005
04_007
04_009
04_013
04_015
04_017
04_019
04_021
04_025
04_027
04_011
04_012
04_023
04_001
05_007
05_015
05_031
05_045
05_051
05_063
05_085
05_115
05_119
05_131
05_143
05_013
05_073
05_095
05_117
05_011
05_021
05_025
05_039
05_049
05_079
05_099
05_101
05_107
05_109
05_001
05_041
05_053
05_065
05_077
05_081
05_097
05_105
05_129
05_135
05_141
05_003
05_037
05_093
05_127
05_017
05_027
05_029
05_075
05_083
05_111
05_133
05_137
05_019
05_043
05_113
05_121
05_147
05_103
05_023
05_055
05_035
05_089

### Get names of districts with recorded yields

In [4]:
# Load Canadian Boundaries for 2017-2022 and 2013-2016
with open('regions_loop_2017.pickle', 'rb') as handle:
    regions_loop_2017 = pickle.load(handle)
with open('regions_loop_2013.pickle', 'rb') as handle:
    regions_loop_2013 = pickle.load(handle)

# Load USA Boundaries
with open('usa_county_polygons.pickle', 'rb') as handle:
    usa_regions = pickle.load(handle)

In [15]:
file_names_yields = []

Corn = []
Cotton = []
Peanuts = []
Soybeans = []
Barley = []
Rice = []
Winter_Wheat = []
Oats = []
Sorghum = []
Spring_Wheat = []
Hay = []
Canola = []
Chickpeas = []
Lentils = []
Peas = []
Flaxseed = []
Sugarbeets = []
Mustard = []
Safflower = []
Triticale = []

In [61]:
# Canada

translationTable = str.maketrans("ÀàÇçÂâÄäÈèÉéÊêËÎîÏÔôÙùÛûÜüŸÿ()", "AaCcAaAaEeEeEeEIiIOoUuUuUuYy--")
crop_yield = pd.read_csv("./32100002_new.csv", usecols=["REF_DATE", "GEO", 'Harvest disposition', "VALUE", "Type of crop"])
# crop_yield = crop_yield[crop_yield["Harvest disposition"] == 'Average yield (bushels per acre)']
crop_yield = crop_yield[crop_yield['VALUE'].notna()]
crop_yield['GEO'] = crop_yield['GEO'].apply(lambda x: x.translate(translationTable)[:75])
crop_names = ['Barley', 'Corn for grain', 'Oats', 'Soybeans', 'Canola', 'Flaxseed', 'Lentils', 'Mustard seed', 
              'Peas, dry', 'Wheat, spring','Wheat, winter remaining', 'Chick peas', 'Triticale']

# 2017-2021
for i, crop_name in enumerate(crop_names):

    if crop_name == 'Canola' or crop_name == 'Peas, dry' or crop_name == 'Mustard seed' or crop_name == 'Lentils' or crop_name == 'Chick peas':
        cyield1 = crop_yield[crop_yield["Harvest disposition"] == 'Average yield (kilograms per hectare)']
        cyield1["VALUE"] = cyield1["VALUE"] * 0.89

    if crop_name != 'Canola' and crop_name != 'Peas, dry' and crop_name != 'Mustard seed' and crop_name != 'Lentils' and crop_name != 'Chick peas':
        cyield1 = crop_yield[crop_yield["Harvest disposition"] == 'Average yield (bushels per acre)']

    cyield1 = cyield1[cyield1["Type of crop"] == crop_name]


    for j, (geometry, name) in enumerate(regions_loop_2017):
        name = name.translate(translationTable)[:75]

        for year in ['2017', '2018', '2019', '2020', '2021', '2022', '2023']:
            cyield = cyield1[cyield1["REF_DATE"] == int(year)][cyield1['GEO'] == name]['VALUE'].tolist()

            if len(cyield) != 0:
                file_name_temp = crop_name+'_'+name+'_'+year
                file_names_yields.append((file_name_temp, cyield[0]))
                
                if crop_name == 'Barley':
                    Barley.append((file_name_temp, cyield[0]))
                if crop_name == 'Corn for grain':
                    Corn.append((file_name_temp, cyield[0]))
                if crop_name == 'Oats':
                    Oats.append((file_name_temp, cyield[0]))
                if crop_name == 'Soybeans':
                    Soybeans.append((file_name_temp, cyield[0]))
                if crop_name == 'Canola':
                    Canola.append((file_name_temp, cyield[0]))
                if crop_name == 'Flaxseed':
                    Flaxseed.append((file_name_temp, cyield[0]))
                if crop_name == 'Lentils':
                    Lentils.append((file_name_temp, cyield[0]))
                if crop_name == 'Mustard seed':
                    Mustard.append((file_name_temp, cyield[0]))
                if crop_name == 'Peas, dry':
                    Peas.append((file_name_temp, cyield[0]))
                if crop_name == 'Wheat, spring':
                    Spring_Wheat.append((file_name_temp, cyield[0]))
                if crop_name == 'Wheat, winter remaining':
                    Winter_Wheat.append((file_name_temp, cyield[0]))
                if crop_name == 'Chick peas':
                    Chickpeas.append((file_name_temp, cyield[0]))
                if crop_name == 'Triticale':
                    Triticale.append((file_name_temp, cyield[0]))
                
                    


# 2013-2016
for i, crop_name in enumerate(crop_names):
    if crop_name == 'Canola' or crop_name == 'Peas, dry' or crop_name == 'Mustard seed' or crop_name == 'Lentils' or crop_name == 'Chick peas':
        cyield1 = crop_yield[crop_yield["Harvest disposition"] == 'Average yield (kilograms per hectare)']
        cyield1["VALUE"] = cyield1["VALUE"] * 0.89

    if crop_name != 'Canola' and crop_name != 'Peas, dry' and crop_name != 'Mustard seed' and crop_name != 'Lentils' and crop_name != 'Chick peas':
        cyield1 = crop_yield[crop_yield["Harvest disposition"] == 'Average yield (bushels per acre)']

    cyield1 = cyield1[cyield1["Type of crop"] == crop_name]


    for j, (geometry, name) in enumerate(regions_loop_2013):
        name = name.translate(translationTable)[:75]

        for year in ['2013', '2014', '2015', '2016']:
            cyield = cyield1[cyield1["REF_DATE"] == int(year)][cyield1['GEO'] == name]['VALUE'].tolist()

            if len(cyield) != 0:
                file_name_temp = crop_name+'_'+name+'_'+year
                file_names_yields.append((file_name_temp, cyield[0]))

                if crop_name == 'Barley':
                    Barley.append((file_name_temp, cyield[0]))
                if crop_name == 'Corn for grain':
                    Corn.append((file_name_temp, cyield[0]))
                if crop_name == 'Oats':
                    Oats.append((file_name_temp, cyield[0]))
                if crop_name == 'Soybeans':
                    Soybeans.append((file_name_temp, cyield[0]))
                if crop_name == 'Canola':
                    Canola.append((file_name_temp, cyield[0]))
                if crop_name == 'Flaxseed':
                    Flaxseed.append((file_name_temp, cyield[0]))
                if crop_name == 'Lentils':
                    Lentils.append((file_name_temp, cyield[0]))
                if crop_name == 'Mustard seed':
                    Mustard.append((file_name_temp, cyield[0]))
                if crop_name == 'Peas, dry':
                    Peas.append((file_name_temp, cyield[0]))
                if crop_name == 'Wheat, spring':
                    Spring_Wheat.append((file_name_temp, cyield[0]))
                if crop_name == 'Wheat, winter remaining':
                    Winter_Wheat.append((file_name_temp, cyield[0]))
                if crop_name == 'Chick peas':
                    Chickpeas.append((file_name_temp, cyield[0]))
                if crop_name == 'Triticale':
                    Triticale.append((file_name_temp, cyield[0]))


In [62]:
print(len(Corn), len(Cotton), len(Peanuts), len(Soybeans), len(Barley), len(Rice), len(Winter_Wheat), len(Oats), 
      len(Sorghum), len(Spring_Wheat), len(Hay), len(Canola), len(Chickpeas), len(Lentils), len(Peas), len(Flaxseed), 
      len(Sugarbeets), len(Mustard), len(Safflower), len(Triticale))

140 0 0 300 515 0 197 506 0 338 0 375 105 199 344 299 0 165 0 97


In [16]:
# USA

crop_yield = pd.read_csv("./usa_county_yields_2012_2023.csv", usecols=["Year", "State", "State ANSI", "County ANSI", "Commodity", "Data Item", "Value"])
crop_yield = crop_yield[crop_yield['Value'].notna()]


crop_names = ['CORN, GRAIN - YIELD, MEASURED IN BU / ACRE',
               'COTTON, UPLAND - YIELD, MEASURED IN LB / ACRE',
               'PEANUTS - YIELD, MEASURED IN LB / ACRE',
               'SOYBEANS - YIELD, MEASURED IN BU / ACRE',
               'BARLEY - YIELD, MEASURED IN BU / ACRE',
               'RICE - YIELD, MEASURED IN LB / ACRE',
               'WHEAT, WINTER - YIELD, MEASURED IN BU / ACRE',
               'OATS - YIELD, MEASURED IN BU / ACRE',
               'SORGHUM, GRAIN - YIELD, MEASURED IN BU / ACRE',
               'WHEAT, SPRING, (EXCL DURUM) - YIELD, MEASURED IN BU / ACRE',
               'HAY, (EXCL ALFALFA) - YIELD, MEASURED IN TONS / ACRE',
               'CANOLA - YIELD, MEASURED IN LB / ACRE',
               'CHICKPEAS - YIELD, MEASURED IN LB / ACRE',
               'LENTILS - YIELD, MEASURED IN LB / ACRE',
               'PEAS, DRY EDIBLE - YIELD, MEASURED IN LB / ACRE',
               'FLAXSEED - YIELD, MEASURED IN BU / ACRE',
               'SUGARBEETS - YIELD, MEASURED IN TONS / ACRE',
#               'SAFFLOWER - YIELD, MEASURED IN LB / ACRE', #Not a lot of data
               'MUSTARD - YIELD, MEASURED IN LB / ACRE']


for i, crop_name in enumerate(crop_names):
    cyield1 = crop_yield[crop_yield["Data Item"] == crop_name]

    for geometry, name in usa_regions:

        for year in range(2012, 2024):
            try:
                cyield = cyield1[cyield1["Year"]==year]
                cyield = cyield[cyield["State ANSI"]==int(name.split('_')[0])]
                cyield = cyield[cyield["County ANSI"]==int(name.split('_')[1])]
                cyield = np.float(cyield['Value'].iloc[0].replace(",", ""))
                file_name_temp = crop_name.replace('/', '-').replace('(', '-').replace(')', '-')+'_'+name[:75]+'_'+str(year)
                
                if crop_name == 'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE':
                    Corn.append((file_name_temp, cyield))
                if crop_name == 'COTTON, UPLAND - YIELD, MEASURED IN LB / ACRE':
                    Cotton.append((file_name_temp, cyield))
                if crop_name == 'PEANUTS - YIELD, MEASURED IN LB / ACRE':
                    Peanuts.append((file_name_temp, cyield))
                if crop_name == 'SOYBEANS - YIELD, MEASURED IN BU / ACRE':
                    Soybeans.append((file_name_temp, cyield))
                if crop_name == 'BARLEY - YIELD, MEASURED IN BU / ACRE':
                    Barley.append((file_name_temp, cyield))
                if crop_name == 'RICE - YIELD, MEASURED IN LB / ACRE':
                    Rice.append((file_name_temp, cyield))    
                if crop_name == 'WHEAT, WINTER - YIELD, MEASURED IN BU / ACRE':
                    Winter_Wheat.append((file_name_temp, cyield))
                if crop_name == 'OATS - YIELD, MEASURED IN BU / ACRE':
                    Oats.append((file_name_temp, cyield))
                if crop_name == 'SORGHUM, GRAIN - YIELD, MEASURED IN BU / ACRE':
                    Sorghum.append((file_name_temp, cyield))
                if crop_name == 'WHEAT, SPRING, (EXCL DURUM) - YIELD, MEASURED IN BU / ACRE':
                    Spring_Wheat.append((file_name_temp, cyield))
                if crop_name == 'HAY, (EXCL ALFALFA) - YIELD, MEASURED IN TONS / ACRE':
                    Hay.append((file_name_temp, cyield))
                if crop_name == 'CANOLA - YIELD, MEASURED IN LB / ACRE':
                    Canola.append((file_name_temp, cyield))
                if crop_name == 'CHICKPEAS - YIELD, MEASURED IN LB / ACRE':
                    Chickpeas.append((file_name_temp, cyield))
                if crop_name == 'LENTILS - YIELD, MEASURED IN LB / ACRE':
                    Lentils.append((file_name_temp, cyield))
                if crop_name == 'PEAS, DRY EDIBLE - YIELD, MEASURED IN LB / ACRE':
                    Peas.append((file_name_temp, cyield))
                if crop_name == 'FLAXSEED - YIELD, MEASURED IN BU / ACRE':
                    Flaxseed.append((file_name_temp, cyield))
                if crop_name == 'SUGARBEETS - YIELD, MEASURED IN TONS / ACRE':
                    Sugarbeets.append((file_name_temp, cyield))
                if crop_name == 'MUSTARD - YIELD, MEASURED IN LB / ACRE':
                    Mustard.append((file_name_temp, cyield))
                if crop_name == 'SAFFLOWER - YIELD, MEASURED IN LB / ACRE':
                    Safflower.append((file_name_temp, cyield))
                
                file_names_yields.append((file_name_temp, cyield))
                
                
            except Exception as e:
#                 print(e)
                pass

KeyboardInterrupt: 

In [66]:
print(len(Corn), len(Cotton), len(Peanuts), len(Soybeans), len(Barley), len(Rice), len(Winter_Wheat), len(Oats), 
      len(Sorghum), len(Spring_Wheat), len(Hay), len(Canola), len(Chickpeas), len(Lentils), len(Peas), len(Flaxseed), 
      len(Sugarbeets), len(Mustard), len(Safflower), len(Triticale))

18038 3675 1191 16283 1942 681 11551 4031 1971 2257 8483 738 179 358 665 428 667 172 0 97


In [67]:
with open('file_names_yields.pickle', 'wb') as file:
    pickle.dump(file_names_yields, file)

In [5]:
with open('file_names_yields.pickle', 'rb') as handle:
    file_names_yields = pickle.load(handle)

In [6]:
just_file_names = [x+'.pickle' for (x,y) in file_names_yields]
print(len(just_file_names))

72938


### (OPTIONAL) Print how many of each are downloaded

In [7]:
files = os.listdir('Raw_Data/86B82C48B82C3957/VI_fesc')
files = [x for x in files if x.endswith('.pickle')]

crops = ["Corn", "Cotton", "Peanuts", "Soybeans", "Barley", "Rice", "Winter", "Oats", 
         "Sorghum", "Spring", "Hay", "Canola", "Chick", "Lentils", "Peas", "Flaxseed", 
         "Sugarbeets", "Mustard", "Safflower", "Triticale"]

for crop in crops:

    cropss = [x for x in files if crop.lower() in x.lower()]

    print(crop, len(cropss))


Corn 14994
Cotton 3075
Peanuts 1002
Soybeans 13512
Barley 1545
Rice 570
Winter 9249
Oats 3262
Sorghum 1677
Spring 1866
Hay 6560
Canola 595
Chick 68
Lentils 282
Peas 602
Flaxseed 345
Sugarbeets 568
Mustard 128
Safflower 0
Triticale 49


#### Canada

In [8]:
import multiprocessing
from natsort import natsorted

files = os.listdir('Raw_Data/TOSHIBA EXT/VI_fesc')
files = [x for x in files if x.endswith('.pickle')]

translationTable = str.maketrans("ÀàÇçÂâÄäÈèÉéÊêËÎîÏÔôÙùÛûÜüŸÿ()", "AaCcAaAaEeEeEeEIiIOoUuUuUuYy--")

# Create inventory image for 2023 because it is not available on Earth Engine
ab_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_AB_v2')
bc_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_BC_v2')
mb_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_MB_v3')
nb_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_NB_v2')
nl_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_NL_v2')
ns_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_NS_v2')
on_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_ON_v2')
pe_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_PE_v2')
qc_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_QC_v2')
sk_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_SK_v2')
yt_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_YT_v2')

def mask_zero(image):
    mask = image.neq(0)  # Create a mask for non-zero values
    return image.updateMask(mask)

# Apply the mask function to each image
ab_2023 = mask_zero(ab_2023)
bc_2023 = mask_zero(bc_2023)
mb_2023 = mask_zero(mb_2023)
nb_2023 = mask_zero(nb_2023)
nl_2023 = mask_zero(nl_2023)
ns_2023 = mask_zero(ns_2023)
on_2023 = mask_zero(on_2023)
pe_2023 = mask_zero(pe_2023)
qc_2023 = mask_zero(qc_2023)
sk_2023 = mask_zero(sk_2023)
yt_2023 = mask_zero(yt_2023)

AAFC_2023 = ee.ImageCollection([ab_2023, bc_2023, mb_2023, nb_2023, nl_2023, 
                                ns_2023, on_2023, pe_2023, qc_2023, sk_2023, yt_2023]).mosaic()


# crop_names = ['Barley', 'Oats', 'Soybeans', 'Canola', 'Flaxseed', 'Peas, dry', 'Wheat, spring', 'Corn for grain']
# crops = [133, 136, 158, 153, 154, 162, 146, 147]

crop_names = ['Barley', 'Corn for grain', 'Oats', 'Soybeans', 'Canola', 'Flaxseed', 'Lentils', 'Mustard seed', 
              'Peas, dry', 'Wheat, spring','Wheat, winter remaining', 'Chick peas', 'Triticale']
crops = [133, 147, 136, 158, 153, 154, 174, 155, 162, 146, 145, 163, 139]


def extract_date(image):
    return ee.Feature(None, {'date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')})

def sample_date(imgcollection, date):
    return imgcollection.filterDate(date).first()

def fesc_band(image):
    NIR = image.select('sur_refl_b02')
    FPAR = image.select('Fpar_500m').add(0.0000001) 
    NDVI = image.select('NDVI')
    Fesc = NIR.multiply(NDVI).divide(FPAR)
    return image.addBands(Fesc.rename('fesc'))


canada_files = []
def process_crop_2017(i, crop):
    # can change geometry somewhere in here to the geometry for the farm
    for j, (geometry, name) in enumerate(regions_loop_2017):
        print(j)
        for year in ['2017', '2018', '2019', '2020', '2021', '2022']:
            try:
                AAFC_mask = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date(year+'-01-01', year+'-12-31')).first()
                AAFC_mask = AAFC_mask.eq(crop)

                MOD15A2H = ee.ImageCollection('MODIS/061/MOD15A2H').filterDate(year+'-01-01', year+'-12-31')
                MOD13Q1 = ee.ImageCollection('MODIS/061/MOD13Q1').filter(ee.Filter.date(year+'-01-01', year+'-12-31'))
                MCD43A4 = ee.ImageCollection('MODIS/061/MCD43A4').filter(ee.Filter.date(year+'-01-01', year+'-12-31'))

                dates = MOD13Q1.map(extract_date).aggregate_array('date').getInfo()
                combined_images = []
                for date in dates:
                    image_MOD13Q1 = sample_date(MOD13Q1, date)
                    image_MOD15A2H = sample_date(MOD15A2H, date)
                    image_MCD43A4 = sample_date(MCD43A4, date)
                    combined_image = image_MOD13Q1.addBands(image_MOD15A2H).addBands(image_MCD43A4)
                    combined_images.append(combined_image)

                veg_reflectance = ee.ImageCollection.fromImages(combined_images)
                veg_reflectance = veg_reflectance.map(fesc_band)            
                veg_reflectance_bands = veg_reflectance.toBands() #-----------------------------------------------------------------------------
                veg_reflectance_bands = veg_reflectance_bands.addBands(veg_reflectance_bands.pixelArea()).unmask(0).updateMask(AAFC_mask) #----- 
            
                try:
                    pixel_values = veg_reflectance_bands.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=250, maxPixels=1e13)
                    dictt = pixel_values.getInfo()
                    num_pixels = len(dictt[list(dictt.keys())[0]])
                except:
                    pixel_values = veg_reflectance_bands.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=500, maxPixels=1e13)
                    dictt = pixel_values.getInfo()
                    num_pixels = len(dictt[list(dictt.keys())[0]])
                    
                if num_pixels > 40000:
                    pixel_values = veg_reflectance_bands.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=500, maxPixels=1e13)
                    dictt = pixel_values.getInfo()
                    num_pixels = len(dictt[list(dictt.keys())[0]]) #----------------------------------------------------
                
                if num_pixels > 0 and num_pixels <= 40000:
                    areas = np.array(dictt['area'])

                    vi_bands = ['BRDF_Albedo_Band_Mandatory_Quality_Band1', 'BRDF_Albedo_Band_Mandatory_Quality_Band2', 
                                 'BRDF_Albedo_Band_Mandatory_Quality_Band3', 'BRDF_Albedo_Band_Mandatory_Quality_Band4', 
                                 'BRDF_Albedo_Band_Mandatory_Quality_Band5', 'BRDF_Albedo_Band_Mandatory_Quality_Band6', 
                                 'BRDF_Albedo_Band_Mandatory_Quality_Band7', 'DayOfYear', 'DetailedQA', 'EVI', 'FparExtra_QC', 
                                 'FparLai_QC', 'FparStdDev_500m', 'Fpar_500m', 'LaiStdDev_500m', 'Lai_500m', 'NDVI', 
                                 'Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band3', 
                                 'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band5', 'Nadir_Reflectance_Band6', 
                                 'Nadir_Reflectance_Band7', 'RelativeAzimuth', 'SolarZenith', 'SummaryQA', 'ViewZenith', 
                                 'fesc', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07']
                    VIs = []
                    for vi_band in vi_bands:
                        keys_with_VI = [key for key in dictt if vi_band in key]
                        sorted_keys_VI = natsorted(keys_with_VI)
                        VI = np.array([dictt[k] for k in sorted_keys_VI]).transpose(1, 0).reshape(-1, 1, 23)
                        VIs.append(VI)
                    VIs = np.concatenate(VIs, axis=1)

                    with open('Raw_Data/TOSHIBA EXT/VI_fesc/'+file_name, 'wb') as file:
                        pickle.dump([VIs, areas], file)
                            
                except Exception as e:
                    print(f"An error occurred: {e}")
                    print(file_name)
                    pass

                
                
            

  
       
if __name__ == "__main__":
    
    # Create a multiprocessing pool
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    
    # Use pool.map to process each crop in parallel
    pool.starmap(process_crop_2017, [(i, crop) for i, crop in enumerate(crops)])

    
    pool.close()
    pool.join()
                
                
                

0000000
0
0
0

0
0
0





11

11

1
12

22
21


11

1
32

3
332


4444
3



2
22

2
55
5

45

666


3
3
3
65

3
7
7
74

8
467

8

44

5
989


87

6
5
5
10
10
85
9

7
9
11
91011


668


612

10
12
1011

9
1313
7

11
1210

11
7
7
1414

12
11
8
1515

12
12
8
8
16
169

13
17
17
149

9
10
18
15
18
19
10
1910

11
16
20
20
21
11
11
2112

17
22
22
12
1218
23

13
23
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
23
22
24
25
23
26
27
28
29
30
31
32
An error occurred: Image.addBands: Parameter 'srcImg' is required.
Peas, dry_Small Area Data Region 1 - Manitoba_2022.pickle
13
An error occurred: Image.addBands: Parameter 'srcImg' is required.
Wheat, spring_Small Area Data Region 1 - Manitoba_2022.pickle
13
An error occurred: Image.addBands: Parameter 'srcImg' is required.
Canola_Small Area Data Region 1 - Manitoba_2022.pickle
13
An error occurred: Image.addBands: Parameter 'srcImg' is required.
Flaxseed_Small Area Data Region 1 - Manitoba_2022.pickle
13
An error occurred: Image.addBands: Par

In [9]:
import multiprocessing
from natsort import natsorted

files = os.listdir('Raw_Data/TOSHIBA EXT/VI_fesc')
files = [x for x in files if x.endswith('.pickle')]

translationTable = str.maketrans("ÀàÇçÂâÄäÈèÉéÊêËÎîÏÔôÙùÛûÜüŸÿ()", "AaCcAaAaEeEeEeEIiIOoUuUuUuYy--")

# Create inventory image for 2023 because it is not available on Earth Engine
ab_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_AB_v2')
bc_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_BC_v2')
mb_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_MB_v3')
nb_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_NB_v2')
nl_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_NL_v2')
ns_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_NS_v2')
on_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_ON_v2')
pe_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_PE_v2')
qc_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_QC_v2')
sk_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_SK_v2')
yt_2023 = ee.Image('projects/ee-amanjotbhullar/assets/aafc_2023/2023_ACI_YT_v2')

def mask_zero(image):
    mask = image.neq(0)  # Create a mask for non-zero values
    return image.updateMask(mask)

# Apply the mask function to each image
ab_2023 = mask_zero(ab_2023)
bc_2023 = mask_zero(bc_2023)
mb_2023 = mask_zero(mb_2023)
nb_2023 = mask_zero(nb_2023)
nl_2023 = mask_zero(nl_2023)
ns_2023 = mask_zero(ns_2023)
on_2023 = mask_zero(on_2023)
pe_2023 = mask_zero(pe_2023)
qc_2023 = mask_zero(qc_2023)
sk_2023 = mask_zero(sk_2023)
yt_2023 = mask_zero(yt_2023)

AAFC_2023 = ee.ImageCollection([ab_2023, bc_2023, mb_2023, nb_2023, nl_2023, 
                                ns_2023, on_2023, pe_2023, qc_2023, sk_2023, yt_2023]).mosaic()


# crop_names = ['Barley', 'Oats', 'Soybeans', 'Canola', 'Flaxseed', 'Peas, dry', 'Wheat, spring', 'Corn for grain']
# crops = [133, 136, 158, 153, 154, 162, 146, 147]

crop_names = ['Barley', 'Corn for grain', 'Oats', 'Soybeans', 'Canola', 'Flaxseed', 'Lentils', 'Mustard seed', 
              'Peas, dry', 'Wheat, spring','Wheat, winter remaining', 'Chick peas', 'Triticale']
crops = [133, 147, 136, 158, 153, 154, 174, 155, 162, 146, 145, 163, 139]


def extract_date(image):
    return ee.Feature(None, {'date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')})

def sample_date(imgcollection, date):
    return imgcollection.filterDate(date).first()

def fesc_band(image):
    NIR = image.select('sur_refl_b02')
    FPAR = image.select('Fpar_500m').add(0.0000001) 
    NDVI = image.select('NDVI')
    Fesc = NIR.multiply(NDVI).divide(FPAR)
    return image.addBands(Fesc.rename('fesc'))

def process_crop_2013(i, crop):
    for j, (geometry, name) in enumerate(regions_loop_2013):
        print(j)
        for year in ['2013', '2014', '2015', '2016']:
            file_name = crop_names[i]+'_'+name.translate(translationTable)[:75]+'_'+year+'.pickle'
            if file_name not in files and file_name in just_file_names:
                try:

                    if year != '2023':
                        AAFC_mask = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date(year+'-01-01', year+'-12-31')).first()
                    if year == '2023':
                        AAFC_mask = AAFC_2023
                    AAFC_mask = AAFC_mask.eq(crop)

                    MOD15A2H = ee.ImageCollection('MODIS/061/MOD15A2H').filterDate(year+'-01-01', year+'-12-31')
                    MOD13Q1 = ee.ImageCollection('MODIS/061/MOD13Q1').filter(ee.Filter.date(year+'-01-01', year+'-12-31'))
                    MCD43A4 = ee.ImageCollection('MODIS/061/MCD43A4').filter(ee.Filter.date(year+'-01-01', year+'-12-31'))

                    dates = MOD13Q1.map(extract_date).aggregate_array('date').getInfo()
                    combined_images = []
                    for date in dates:
                        image_MOD13Q1 = sample_date(MOD13Q1, date)
                        image_MOD15A2H = sample_date(MOD15A2H, date)
                        image_MCD43A4 = sample_date(MCD43A4, date)
                        combined_image = image_MOD13Q1.addBands(image_MOD15A2H).addBands(image_MCD43A4)
                        combined_images.append(combined_image)

                    veg_reflectance = ee.ImageCollection.fromImages(combined_images)
                    veg_reflectance = veg_reflectance.map(fesc_band)
                    veg_reflectance_bands = veg_reflectance.toBands() #-----------------------------------------------------------------------------
                    veg_reflectance_bands = veg_reflectance_bands.addBands(veg_reflectance_bands.pixelArea()).unmask(0).updateMask(AAFC_mask) #-----
                    
                    try:
                        pixel_values = veg_reflectance_bands.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=250, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]])
                    except:
                        pixel_values = veg_reflectance_bands.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=500, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]])
                        
                    if num_pixels > 40000:
                        pixel_values = veg_reflectance_bands.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=500, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]]) #----------------------------------------------------
                    
                    
                    if num_pixels > 0 and num_pixels <= 40000:
                        areas = np.array(dictt['area'])

                        vi_bands = ['BRDF_Albedo_Band_Mandatory_Quality_Band1', 'BRDF_Albedo_Band_Mandatory_Quality_Band2', 
                                     'BRDF_Albedo_Band_Mandatory_Quality_Band3', 'BRDF_Albedo_Band_Mandatory_Quality_Band4', 
                                     'BRDF_Albedo_Band_Mandatory_Quality_Band5', 'BRDF_Albedo_Band_Mandatory_Quality_Band6', 
                                     'BRDF_Albedo_Band_Mandatory_Quality_Band7', 'DayOfYear', 'DetailedQA', 'EVI', 'FparExtra_QC', 
                                     'FparLai_QC', 'FparStdDev_500m', 'Fpar_500m', 'LaiStdDev_500m', 'Lai_500m', 'NDVI', 
                                     'Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band3', 
                                     'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band5', 'Nadir_Reflectance_Band6', 
                                     'Nadir_Reflectance_Band7', 'RelativeAzimuth', 'SolarZenith', 'SummaryQA', 'ViewZenith', 
                                     'fesc', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07']
                        VIs = []
                        for vi_band in vi_bands:
                            keys_with_VI = [key for key in dictt if vi_band in key]
                            sorted_keys_VI = natsorted(keys_with_VI)
                            VI = np.array([dictt[k] for k in sorted_keys_VI]).transpose(1, 0).reshape(-1, 1, 23)
                            VIs.append(VI)
                        VIs = np.concatenate(VIs, axis=1)

                        with open('Raw_Data/TOSHIBA EXT/VI_fesc/'+file_name, 'wb') as file:
                            pickle.dump([VIs, areas], file)
                            
                except Exception as e:
                    print(f"An error occurred: {e}")
                    print(file_name)
                    pass
                
                
                
if __name__ == "__main__":
    
    # Create a multiprocessing pool
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    
    # Use pool.map to process each crop in parallel
    pool.starmap(process_crop_2013, [(i, crop) for i, crop in enumerate(crops)])

    
    pool.close()
    pool.join()

0000000





0
0

1
1
11

11

1
00
2

2
22

2
3
4
3
331


3
150


2
40
44


63

42

4
7
51

55

3
5
2
58

6
3
664


6
1
97

4
6
55

10
87

77

6
2
9
11
6
77

8
10
8
12
87
8

11
3
9
13
8
9
812

9
1014

9
13
9
9410


11
15
10
14
12
10
16
15
1011

10
115

1617

11
1712

18
1111
12

12
618

19
13
19
1212

20
7
20
14
1321

21
8
15
22
22
14
16
9
2323

17
15
10
18
16
1119

17
20
12
2118

22
19
23
20
21
22
23
An error occurred: Image.addBands: Parameter 'srcImg' is required.
Barley_Bas-Saint-Laurent and Gaspesie-Iles de-la-Madeleine, 1 - Quebec_2016.pickle
1
An error occurred: Image.addBands: Parameter 'srcImg' is required.
Oats_Bas-Saint-Laurent and Gaspesie-Iles de-la-Madeleine, 1 - Quebec_2016.pickle
1
An error occurred: Image.addBands: Parameter 'srcImg' is required.
Soybeans_Saguenay-Lac-Saint-Jean and Cote-Nord, 2 - Quebec_2016.pickle
2
An error occurred: Image.addBands: Parameter 'srcImg' is required.
Corn for grain_Saguenay-Lac-Saint-Jean and Cote-Nord, 2 - Quebec_2016.pickle
2
An erro

USA

In [28]:
import multiprocessing
from natsort import natsorted

files = os.listdir('Raw_Data/TOSHIBA EXT/VI_fesc')
files = [x for x in files if x.endswith('.pickle')]

# crop_names = ['BARLEY - YIELD, MEASURED IN BU / ACRE',
#                  'CANOLA - YIELD, MEASURED IN LB / ACRE',
#                  'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE',
#                  'COTTON, UPLAND - YIELD, MEASURED IN LB / ACRE',
#                  'FLAXSEED - YIELD, MEASURED IN BU / ACRE',
#                  'HAY, (EXCL ALFALFA) - YIELD, MEASURED IN TONS / ACRE',
#                  'LENTILS - YIELD, MEASURED IN LB / ACRE',
#                  'OATS - YIELD, MEASURED IN BU / ACRE',
#                  'PEAS, DRY EDIBLE - YIELD, MEASURED IN LB / ACRE',
#                  'POTATOES - YIELD, MEASURED IN CWT / ACRE',
#                  'RICE - YIELD, MEASURED IN LB / ACRE',
#                  'SORGHUM, GRAIN - YIELD, MEASURED IN BU / ACRE',
#                  'SOYBEANS - YIELD, MEASURED IN BU / ACRE',
#                  'SUGARBEETS - YIELD, MEASURED IN TONS / ACRE',
#                  'WHEAT, SPRING, (EXCL DURUM) - YIELD, MEASURED IN BU / ACRE',
#                  'WHEAT, SPRING, DURUM - YIELD, MEASURED IN BU / ACRE',
#                  'WHEAT, WINTER - YIELD, MEASURED IN BU / ACRE']


# crops = [21, 31, 1, 2, 32, 37, 52, 28, 53, 43, 3, 4, 5, 41, 23, 22, 24]



crop_names = ['CORN, GRAIN - YIELD, MEASURED IN BU / ACRE',
               'COTTON, UPLAND - YIELD, MEASURED IN LB / ACRE',
               'PEANUTS - YIELD, MEASURED IN LB / ACRE',
               'SOYBEANS - YIELD, MEASURED IN BU / ACRE',
               'BARLEY - YIELD, MEASURED IN BU / ACRE',
               'RICE - YIELD, MEASURED IN LB / ACRE',
               'WHEAT, WINTER - YIELD, MEASURED IN BU / ACRE',
               'OATS - YIELD, MEASURED IN BU / ACRE',
               'SORGHUM, GRAIN - YIELD, MEASURED IN BU / ACRE',
               'WHEAT, SPRING, (EXCL DURUM) - YIELD, MEASURED IN BU / ACRE',
               'HAY, (EXCL ALFALFA) - YIELD, MEASURED IN TONS / ACRE',
               'CANOLA - YIELD, MEASURED IN LB / ACRE',
               'CHICKPEAS - YIELD, MEASURED IN LB / ACRE',
               'LENTILS - YIELD, MEASURED IN LB / ACRE',
               'PEAS, DRY EDIBLE - YIELD, MEASURED IN LB / ACRE',
               'FLAXSEED - YIELD, MEASURED IN BU / ACRE',
               'SUGARBEETS - YIELD, MEASURED IN TONS / ACRE',
#               'SAFFLOWER - YIELD, MEASURED IN LB / ACRE', # not enough data
               'MUSTARD - YIELD, MEASURED IN LB / ACRE']




crops = [1, 2, 10, 5, 21, 3, 24, 28, 4, 23, 37, 31, 51, 52, 53, 32, 41, 35]






def extract_date(image):
    return ee.Feature(None, {'date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')})

def sample_date(imgcollection, date):
    return imgcollection.filterDate(date).first()

def fesc_band(image):
    NIR = image.select('sur_refl_b02')
    FPAR = image.select('Fpar_500m').add(0.0000001) 
    NDVI = image.select('NDVI')
    Fesc = NIR.multiply(NDVI).divide(FPAR)
    return image.addBands(Fesc.rename('fesc'))




def process_crop(i, crop):
    for j, (geometry, name) in enumerate(usa_regions):
#         print(j)
        for year in range(2012, 2024):
            year = str(year)
            file_name = crop_names[i].replace('/', '-').replace('(', '-').replace(')', '-')+'_'+name[:75]+'_'+year+'.pickle'
            if file_name not in files and file_name in just_file_names:
                try:
                        
                    # Mask for crop
                    cropLandcover = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(year+'-01-01', year+'-12-31')).first().select('cropland')
                    mask = cropLandcover.eq(crop)

                    MOD15A2H = ee.ImageCollection('MODIS/061/MOD15A2H').filterDate(year+'-01-01', year+'-12-31')
                    MOD13Q1 = ee.ImageCollection('MODIS/061/MOD13Q1').filter(ee.Filter.date(year+'-01-01', year+'-12-31'))
                    MCD43A4 = ee.ImageCollection('MODIS/061/MCD43A4').filter(ee.Filter.date(year+'-01-01', year+'-12-31'))

                    dates = MOD13Q1.map(extract_date).aggregate_array('date').getInfo()
                    combined_images = []
                    for date in dates:
                        image_MOD13Q1 = sample_date(MOD13Q1, date)
                        image_MOD15A2H = sample_date(MOD15A2H, date)
                        image_MCD43A4 = sample_date(MCD43A4, date)
                        combined_image = image_MOD13Q1.addBands(image_MOD15A2H).addBands(image_MCD43A4)
                        combined_images.append(combined_image)

                    veg_reflectance = ee.ImageCollection.fromImages(combined_images)
                    veg_reflectance = veg_reflectance.map(fesc_band)
                    veg_reflectance_bands = veg_reflectance.toBands() #-----------------------------------------------------------------------------
                    veg_reflectance_bands = veg_reflectance_bands.addBands(veg_reflectance_bands.pixelArea()).unmask(0).updateMask(mask) #-----
                    
                    try:
                        pixel_values = veg_reflectance_bands.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry.geometry(), scale=250, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]])
                    except:
                        pixel_values = veg_reflectance_bands.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry.geometry(), scale=500, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]])
                        
                    if num_pixels > 40000:
                        pixel_values = veg_reflectance_bands.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry.geometry(), scale=500, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]]) #----------------------------------------------------
                    
                    
                    if num_pixels > 0 and num_pixels <= 40000:
                        areas = np.array(dictt['area'])

                        vi_bands = ['BRDF_Albedo_Band_Mandatory_Quality_Band1', 'BRDF_Albedo_Band_Mandatory_Quality_Band2', 
                                     'BRDF_Albedo_Band_Mandatory_Quality_Band3', 'BRDF_Albedo_Band_Mandatory_Quality_Band4', 
                                     'BRDF_Albedo_Band_Mandatory_Quality_Band5', 'BRDF_Albedo_Band_Mandatory_Quality_Band6', 
                                     'BRDF_Albedo_Band_Mandatory_Quality_Band7', 'DayOfYear', 'DetailedQA', 'EVI', 'FparExtra_QC', 
                                     'FparLai_QC', 'FparStdDev_500m', 'Fpar_500m', 'LaiStdDev_500m', 'Lai_500m', 'NDVI', 
                                     'Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band3', 
                                     'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band5', 'Nadir_Reflectance_Band6', 
                                     'Nadir_Reflectance_Band7', 'RelativeAzimuth', 'SolarZenith', 'SummaryQA', 'ViewZenith', 
                                     'fesc', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07']
                        VIs = []
                        for vi_band in vi_bands:
                            keys_with_VI = [key for key in dictt if vi_band in key]
                            sorted_keys_VI = natsorted(keys_with_VI)
                            VI = np.array([dictt[k] for k in sorted_keys_VI]).transpose(1, 0).reshape(-1, 1, 23)
                            VIs.append(VI)
                        VIs = np.concatenate(VIs, axis=1)
                        
#                         with open('Raw_Data/TOSHIBA EXT/VI_fesc/'+file_name, 'wb') as file:
#                             pickle.dump([VIs, areas], file)

                        with open('Raw_Data/86B82C48B82C3957/VI_fesc/'+file_name, 'wb') as file:
                            pickle.dump([VIs, areas], file)
                            
                            
                            
                except Exception as e:
                    print(f"An error occurred: {e}")
                    print(file_name)
                    pass
            
        

if __name__ == "__main__":
    
    # Create a multiprocessing pool
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    
    # Use pool.map to process each crop in parallel
    pool.starmap(process_crop, [(i, crop) for i, crop in enumerate(crops)])
    
    pool.close()
    pool.join()




An error occurred: Image.addBands: Parameter 'srcImg' is required.
CORN, GRAIN - YIELD, MEASURED IN BU - ACRE_01_003_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
SOYBEANS - YIELD, MEASURED IN BU - ACRE_01_003_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
PEANUTS - YIELD, MEASURED IN LB - ACRE_01_003_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
COTTON, UPLAND - YIELD, MEASURED IN LB - ACRE_01_001_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
CORN, GRAIN - YIELD, MEASURED IN BU - ACRE_01_003_2022.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
PEANUTS - YIELD, MEASURED IN LB - ACRE_01_003_2022.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
SOYBEANS - YIELD, MEASURED IN BU - ACRE_01_003_2022.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
COTTON, UPLAND - YIELD, MEASURED IN LB - 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_31_159_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_31_067_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_31_067_2022.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_31_099_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_31_099_2022.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_31_033_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_31_033_2022.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_31_161_2016.pickle
An error occurred:

Process ForkPoolWorker-109:
Process ForkPoolWorker-114:
Process ForkPoolWorker-106:
Process ForkPoolWorker-113:
Process ForkPoolWorker-104:
Process ForkPoolWorker-112:
Process ForkPoolWorker-99:
Process ForkPoolWorker-98:
Process ForkPoolWorker-111:
Process ForkPoolWorker-101:
Process ForkPoolWorker-131:
Process ForkPoolWorker-138:
Process ForkPoolWorker-108:
Process ForkPoolWorker-105:
Process ForkPoolWorker-102:
Process ForkPoolWorker-133:


KeyboardInterrupt: 

Process ForkPoolWorker-119:
Process ForkPoolWorker-118:
Process ForkPoolWorker-116:
Process ForkPoolWorker-136:
Process ForkPoolWorker-124:
Process ForkPoolWorker-142:
Process ForkPoolWorker-144:
Process ForkPoolWorker-129:
Process ForkPoolWorker-128:
Process ForkPoolWorker-140:
Process ForkPoolWorker-137:
Process ForkPoolWorker-123:
Process ForkPoolWorker-110:
Process ForkPoolWorker-139:
Process ForkPoolWorker-135:
Process ForkPoolWorker-117:
Process ForkPoolWorker-130:
Process ForkPoolWorker-121:
Process ForkPoolWorker-122:
Process ForkPoolWorker-127:
Process ForkPoolWorker-134:
Process ForkPoolWorker-115:
Process ForkPoolWorker-141:
Process ForkPoolWorker-132:
Process ForkPoolWorker-120:
Process ForkPoolWorker-126:
Process ForkPoolWorker-125:
Process ForkPoolWorker-143:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceb

An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_40_073_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_40_073_2022.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_40_085_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_40_053_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_40_053_2022.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_40_059_2016.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - YIELD, MEASURED IN BU - ACRE_40_059_2022.pickle
An error occurred: Image.addBands: Parameter 'srcImg' is required.
WHEAT, WINTER - 

### 3. Download Climate Variables

### Fast Downloads for Climate/Environment and New Variables
New variables include time series of inventory and satellite imagery

In [4]:
# Load Canadian Boundaries for 2017-2022 and 2013-2016
with open('regions_loop_2017.pickle', 'rb') as handle:
    regions_loop_2017 = pickle.load(handle)
with open('regions_loop_2013.pickle', 'rb') as handle:
    regions_loop_2013 = pickle.load(handle)

# Load USA Boundaries
with open('usa_county_polygons.pickle', 'rb') as handle:
    usa_regions = pickle.load(handle)

In [5]:
with open('file_names_yields.pickle', 'rb') as handle:
    file_names_yields = pickle.load(handle)

In [6]:
just_file_names = [x+'.pickle' for (x,y) in file_names_yields]
print(len(just_file_names))

73194


#### Canada

In [90]:
import sys
import multiprocessing
from natsort import natsorted
from sklearn.preprocessing import OneHotEncoder

new_limit = 15000
sys.setrecursionlimit(new_limit)

translationTable = str.maketrans("ÀàÇçÂâÄäÈèÉéÊêËÎîÏÔôÙùÛûÜüŸÿ()", "AaCcAaAaEeEeEeEIiIOoUuUuUuYy--")


files = os.listdir('Raw_Data/TOSHIBA EXT/environment_ndvi_district/CA/')
files = [x for x in files if x.endswith('.pickle')]


crop_names = ['Barley', 'Corn for grain', 'Oats', 'Soybeans', 'Canola', 'Flaxseed', 'Lentils', 'Mustard seed', 
              'Peas, dry', 'Wheat, spring','Wheat, winter remaining', 'Chick peas', 'Triticale']
crops = [133, 147, 136, 158, 153, 154, 174, 155, 162, 146, 145, 163, 139]



elevation = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').select('DSM')
proj_slope = elevation.first().select(0).projection()
slope = ee.Terrain.slope(elevation.mosaic().setDefaultProjection(proj_slope))
aspect = ee.Terrain.aspect(elevation.mosaic().setDefaultProjection(proj_slope))

soil_water_content = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01")
original_band_names = soil_water_content.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('soil_water_content_').cat(name))
soil_water_content = soil_water_content.select(original_band_names, new_band_names)

oc = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02")
original_band_names = oc.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('oc_').cat(name))
oc = oc.select(original_band_names, new_band_names)

pH = ee.Image("OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02")
original_band_names = pH.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('pH_').cat(name))
pH = pH.select(original_band_names, new_band_names)

bulk = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02")
original_band_names = bulk.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('bulk_').cat(name))
bulk = bulk.select(original_band_names, new_band_names)

texture = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")
original_band_names = texture.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('texture_').cat(name))
texture = texture.select(original_band_names, new_band_names)

# Global map of lands that irrigate. This map is assumed to not change with time, which is wrong.
irrigation = ee.Image('USGS/GFSAD1000_V1').select('landcover').rename('irrigation')
# irrigation_major = crop_irrigation.eq(1)
# irrigation_minor = crop_irrigation.eq(2)
# irrigation = irrigation_major.Or(irrigation_minor)


def process_crop(i, crop):
    for j, (geometry, name) in enumerate(regions_loop_2017):
#         print(j)
        for year in ['2017', '2018', '2019', '2020', '2021', '2022', '2023']:
            file_name = crop_names[i]+'_'+name.translate(translationTable)[:75]+'_'+year+'.pickle'
            if file_name not in files and file_name in just_file_names:
                try:
    
                    masked_soil_clim_land_image = ee.Image()

                    AAFC = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date(year+'-01-01', year+'-12-31')).first()
                    mask = AAFC.eq(crop)


                    # Previous Year Crops
                    for y in range(8):  # Loop through the previous 8 years
                        prev_year = int(year) - y
                        if prev_year >= 2013:
                            start_date = str(prev_year) + '-01-01'
                            end_date = str(prev_year) + '-12-31'
                            yearly_image = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date(start_date, end_date)).first()
                        else:
                            yearly_image = ee.Image.constant(0).rename('landcover')
                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([yearly_image])





                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([slope])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([elevation.mosaic()])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([aspect])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([soil_water_content])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([oc])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([pH])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([bulk])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([texture])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([irrigation]) #irrigation status


                            
                    # Temporal Covariates
                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_prec = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
                                          .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                 ee.Date(year + '-' + month + '-' + str(week_end)))) \
                                          .select('precipitationCal') \
                                          .sum()  # Get the max precipitation for the week
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_prec.rename(f'precip_sum')])
                             
                        
                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for day in range(1, 28 + 1, 7): 
                            week_end = day + 6
                            week_temp_collection = ee.ImageCollection([])
                            for d in range(day, week_end + 1):
                                day_temp = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                            .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(d)).getRange('day')))

                                day_temp_max = day_temp.select('Tair_f_inst').max()
                                day_temp_min = day_temp.select('Tair_f_inst').min()
                                day_temp_diurnal = day_temp_max.subtract(day_temp_min)
                                week_temp_collection = week_temp_collection.merge(ee.ImageCollection([day_temp_diurnal]))

                            week_temp_max_diurnal = week_temp_collection.mean()
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_temp_max_diurnal.rename(f'diurnal_temp_mean')])

                            
                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_temp = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                          .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                 ee.Date(year + '-' + month + '-' + str(week_end)))) \
                                          .select('Tair_f_inst') \
                                          .max() 
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_temp.rename(f'Tair_f_inst_max')])


                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                            .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                   ee.Date(year + '-' + month + '-' + str(week_end))))
                            week_humidity = week_GLDAS.select('Qair_f_inst').mean()
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_humidity.rename(f'Qair_f_inst_mean')])


                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                            .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                   ee.Date(year + '-' + month + '-' + str(week_end))))
                            week_long_radiation = week_GLDAS.select('LWdown_f_tavg').sum()
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_long_radiation.rename(f'LWdown_f_tavg_sum')])
               

                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                            .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                   ee.Date(year + '-' + month + '-' + str(week_end))))
                            week_short_radiation = week_GLDAS.select('SWdown_f_tavg').sum()
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_short_radiation.rename(f'SWdown_f_tavg_sum')])

                            
                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_SMAP = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
                                          .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                 ee.Date(year + '-' + month + '-' + str(week_end))))

                            if week_SMAP.size().getInfo() != 0:
                                week_soil_moisture_top = week_SMAP.select('sm_surface').mean()
                            else:
                                week_soil_moisture_top = ee.Image.constant(0)
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_soil_moisture_top.rename(f'sm_surface_mean')])
                            
                            
                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_SMAP = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
                                          .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                 ee.Date(year + '-' + month + '-' + str(week_end))))

                            if week_SMAP.size().getInfo() != 0:
                                week_soil_moisture_root = week_SMAP.select('sm_rootzone').mean()
                            else:
                                week_soil_moisture_root = ee.Image.constant(0)
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_soil_moisture_root.rename(f'sm_rootzone_mean')])


                            
                            
                            
                    

                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands(masked_soil_clim_land_image.pixelArea())
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands(masked_soil_clim_land_image.pixelLonLat())
                    masked_soil_clim_land_image = masked_soil_clim_land_image.unmask(0).updateMask(mask)

                    
                    try:
                        pixel_values = masked_soil_clim_land_image.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=500, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]])
                    except:
                        pixel_values = masked_soil_clim_land_image.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=1000, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]])
                        
                    if num_pixels > 40000:
                        pixel_values = masked_soil_clim_land_image.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=1000, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]])

                    
                    if num_pixels > 0 and num_pixels <= 40000:

                        # Texture ----------------------------------------------------------------------------------------------------
                        data_labels = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]).reshape(-1, 1)
                        encoder = OneHotEncoder(sparse=False).fit(data_labels)

                        keys_with_texture = [key for key in dictt if 'texture' in key]
                        sorted_keys_texture = natsorted(keys_with_texture)
                        texture_v = np.array([dictt[k] for k in sorted_keys_texture]).transpose(1, 0)
                        non_zero_rows = ~np.all(texture_v == 0, axis=1)
                        texture_v = texture_v[non_zero_rows]
                        texture_v = encoder.transform(texture_v.reshape(-1, 1)).reshape(-1, 6, 12).transpose(0, 2, 1)
    #                     print('texture', texture_v.shape)


                        # Area, location --------------------------------------------------------------------------------------
                        area = np.array(dictt['area'])[non_zero_rows]
                        longitude = np.array(dictt['longitude'])[non_zero_rows]
                        latitude = np.array(dictt['latitude'])[non_zero_rows]
                        #print('area, lon, lat', area.shape, longitude.shape, latitude.shape)

                        # Time-Dependent -------------------------------------------------------------------------------------------------
                        keys_with_sm_surface = [key for key in dictt if 'sm_surface_mean' in key]
                        sorted_keys_sm_surface = natsorted(keys_with_sm_surface)
                        sm_surface_v = np.array([dictt[k] for k in sorted_keys_sm_surface]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_sm_rootzone = [key for key in dictt if 'sm_rootzone_mean' in key]
                        sorted_keys_sm_rootzone = natsorted(keys_with_sm_rootzone)
                        sm_rootzone_v = np.array([dictt[k] for k in sorted_keys_sm_rootzone]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_SWdown_f_tavg = [key for key in dictt if 'SWdown_f_tavg_sum' in key]
                        sorted_keys_SWdown_f_tavg = natsorted(keys_with_SWdown_f_tavg)
                        SWdown_f_tavg_v = np.array([dictt[k] for k in sorted_keys_SWdown_f_tavg]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_LWdown_f_tavg = [key for key in dictt if 'LWdown_f_tavg_sum' in key]
                        sorted_keys_LWdown_f_tavg = natsorted(keys_with_LWdown_f_tavg)
                        LWdown_f_tavg_v = np.array([dictt[k] for k in sorted_keys_LWdown_f_tavg]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_Qair_f_inst = [key for key in dictt if 'Qair_f_inst_mean' in key]
                        sorted_keys_Qair_f_inst = natsorted(keys_with_Qair_f_inst)
                        Qair_f_inst_v = np.array([dictt[k] for k in sorted_keys_Qair_f_inst]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_Tair_f_inst_max = [key for key in dictt if 'Tair_f_inst_max' in key]
                        sorted_keys_Tair_f_inst_max = natsorted(keys_with_Tair_f_inst_max)
                        Tair_f_inst_max_v = np.array([dictt[k] for k in sorted_keys_Tair_f_inst_max]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_Tair_f_inst = [key for key in dictt if 'diurnal_temp_mean' in key]
                        sorted_keys_Tair_f_inst = natsorted(keys_with_Tair_f_inst)
                        Tair_f_inst_v = np.array([dictt[k] for k in sorted_keys_Tair_f_inst]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_precipitationCal = [key for key in dictt if 'precip_sum' in key]
                        sorted_keys_precipitationCal = natsorted(keys_with_precipitationCal)
                        precipitationCal_v = np.array([dictt[k] for k in sorted_keys_precipitationCal]).transpose(1, 0).reshape(-1, 1, 48)

                        time_dependent = np.concatenate([sm_surface_v, sm_rootzone_v, SWdown_f_tavg_v, LWdown_f_tavg_v, Qair_f_inst_v, Tair_f_inst_max_v, Tair_f_inst_v, precipitationCal_v], axis=1)
                        time_dependent = time_dependent[non_zero_rows]
    #                     print('time_dependent', time_dependent.shape)

                        # Soil -------------------------------------------------------------------------------------------------
                        keys_with_pH = [key for key in dictt if 'pH' in key]
                        sorted_keys_pH = natsorted(keys_with_pH)
                        pH_v = np.array([dictt[k] for k in sorted_keys_pH]).transpose(1, 0).reshape(-1, 1, 6)

                        keys_with_soil_water_content = [key for key in dictt if 'soil_water_content' in key]
                        sorted_keys_soil_water_content = natsorted(keys_with_soil_water_content)
                        soil_water_content_v = np.array([dictt[k] for k in sorted_keys_soil_water_content]).transpose(1, 0).reshape(-1, 1, 6)

                        keys_with_oc = [key for key in dictt if 'oc' in key]
                        sorted_keys_oc = natsorted(keys_with_oc)
                        oc_v = np.array([dictt[k] for k in sorted_keys_oc]).transpose(1, 0).reshape(-1, 1, 6)

                        keys_with_bulk = [key for key in dictt if 'bulk' in key]
                        sorted_keys_bulk = natsorted(keys_with_bulk)
                        bulk_v = np.array([dictt[k] for k in sorted_keys_bulk]).transpose(1, 0).reshape(-1, 1, 6)

                        soil = np.concatenate([pH_v, soil_water_content_v, oc_v, bulk_v], axis=1)
                        soil = soil[non_zero_rows]
                        #print('soil', soil.shape)

                        # Singles ------------------------------------------------------------------------------------------------------
                        keys_with_slope = [key for key in dictt if 'slope' in key]
                        sorted_keys_slope = natsorted(keys_with_slope)
                        slope_v = np.array([dictt[k] for k in sorted_keys_slope]).transpose(1, 0).reshape(-1, 1)

                        keys_with_aspect = [key for key in dictt if 'aspect' in key]
                        sorted_keys_aspect = natsorted(keys_with_aspect)
                        aspect_v = np.array([dictt[k] for k in sorted_keys_aspect]).transpose(1, 0).reshape(-1, 1)

                        keys_with_DSM = [key for key in dictt if 'DSM' in key]
                        sorted_keys_DSM = natsorted(keys_with_DSM)
                        DSM_v = np.array([dictt[k] for k in sorted_keys_DSM]).transpose(1, 0).reshape(-1, 1)


                        data_labels = np.array([0, 1, 2, 3, 4, 5]).reshape(-1, 1)
                        encoder = OneHotEncoder(sparse=False).fit(data_labels)
                        keys_with_irrigation = [key for key in dictt if 'irrigation' in key]
                        sorted_keys_irrigation = natsorted(keys_with_irrigation)
                        irrigation_v = np.array([dictt[k] for k in sorted_keys_irrigation]).transpose(1, 0)
                        irrigation_v = encoder.transform(irrigation_v.reshape(-1, 1)).reshape(-1, 6)

#                         singles = np.concatenate([slope_v, aspect_v, DSM_v, irrigation_v], axis=1)
                        singles = np.concatenate([slope_v, aspect_v, DSM_v], axis=1)
                        singles = singles[non_zero_rows]
                        #print('singles', singles.shape)

                        # Time series inventory-------------------------------------------------------------------------------------------
                        conversion_dict = {
                            0: 0, # mask
                            145: 24,  # Winter Wheat
                            174: 52,  # Lentils
                            147: 1,   # Corn
                            133: 21,  # Barley
                            142: 4,   # Sorghum
                            163: 51,  # Chickpeas
                            178: 41,  # Sugarbeets
                            155: 35,  # Mustard
                            136: 28,  # Oats
                            158: 5,   # Soy
                            153: 31,  # Canola
                            154: 32,  # Flaxseed
                            162: 53,  # Peas
                            146: 23,  # Spring Wheat
                            122: 176, # Grassland/Pasture
                        }
                        valid_values = {0, 1, 2, 3, 4, 5, 21, 22, 23, 24, 28, 31, 32,
                                        35, 36, 37, 41, 42, 43, 51, 52, 53, 141, 61, 176}
                        data_labels = np.array([0, 1, 2, 3, 4, 5, 21, 22, 23, 24, 28, 31, 32,
                                                35, 36, 37, 41, 42, 43, 51, 52, 53, 141, 61, 176, 999]).reshape(-1, 1)
                        encoder = OneHotEncoder(sparse=False).fit(data_labels)

                        keys_with_landcover = [key for key in dictt if 'landcover' in key]
                        sorted_keys_landcover = natsorted(keys_with_landcover)
                        landcover_v = np.array([dictt[k] for k in sorted_keys_landcover]).transpose(1, 0)
                        landcover_v = landcover_v[non_zero_rows]
                        landcover_v = np.array([conversion_dict.get(x, 999) if conversion_dict.get(x, 999) in valid_values else 999 for x in landcover_v.reshape(-1)]).reshape(-1, 8)
                        landcover_v = encoder.transform(landcover_v.reshape(-1, 1)).reshape(-1, 8, 26).transpose(0, 2, 1)

                        landcover_v = landcover_v[:, 1:, :]
                        #print('landcover', landcover_v.shape)


                        arrays = [area.astype(np.float32), longitude, latitude, time_dependent.astype(np.float32), soil.astype(np.int32), texture_v.astype(np.int32), singles.astype(np.float32), landcover_v.astype(np.int32)]
                        print(area.shape, longitude.shape, latitude.shape, time_dependent.shape, 
                              soil.shape, texture_v.shape, singles.shape, landcover_v.shape)                    

                        with open('Raw_Data/TOSHIBA EXT/environment_ndvi_district/CA/'+file_name, 'wb') as file:
                            pickle.dump(arrays, file)
                            print(file_name)
                        
                except Exception as e:
                    print(f"An error occurred: {e}")
                    print(file_name)
                    pass




if __name__ == "__main__":
    
    # Create a multiprocessing pool
#     pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    pool = multiprocessing.Pool(processes=30)

    
    # Use pool.map to process each crop in parallel
    pool.starmap(process_crop, [(i, crop) for i, crop in enumerate(crops)])
    
    pool.close()
    pool.join()
              
            


    
##print('Done!')

Canada 2016

In [11]:
import sys
import multiprocessing
from natsort import natsorted
from sklearn.preprocessing import OneHotEncoder

new_limit = 15000
sys.setrecursionlimit(new_limit)

translationTable = str.maketrans("ÀàÇçÂâÄäÈèÉéÊêËÎîÏÔôÙùÛûÜüŸÿ()", "AaCcAaAaEeEeEeEIiIOoUuUuUuYy--")


files = os.listdir('Raw_Data/TOSHIBA EXT/environment_ndvi_district/CA/')
files = [x for x in files if x.endswith('.pickle')]


crop_names = ['Barley', 'Corn for grain', 'Oats', 'Soybeans', 'Canola', 'Flaxseed', 'Lentils', 'Mustard seed', 
              'Peas, dry', 'Wheat, spring','Wheat, winter remaining', 'Chick peas', 'Triticale']
crops = [133, 147, 136, 158, 153, 154, 174, 155, 162, 146, 145, 163, 139]



elevation = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').select('DSM')
proj_slope = elevation.first().select(0).projection()
slope = ee.Terrain.slope(elevation.mosaic().setDefaultProjection(proj_slope))
aspect = ee.Terrain.aspect(elevation.mosaic().setDefaultProjection(proj_slope))

soil_water_content = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01")
original_band_names = soil_water_content.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('soil_water_content_').cat(name))
soil_water_content = soil_water_content.select(original_band_names, new_band_names)

oc = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02")
original_band_names = oc.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('oc_').cat(name))
oc = oc.select(original_band_names, new_band_names)

pH = ee.Image("OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02")
original_band_names = pH.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('pH_').cat(name))
pH = pH.select(original_band_names, new_band_names)

bulk = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02")
original_band_names = bulk.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('bulk_').cat(name))
bulk = bulk.select(original_band_names, new_band_names)

texture = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")
original_band_names = texture.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('texture_').cat(name))
texture = texture.select(original_band_names, new_band_names)

# Global map of lands that irrigate. This map is assumed to not change with time, which is wrong.
irrigation = ee.Image('USGS/GFSAD1000_V1').select('landcover').rename('irrigation')
# irrigation_major = crop_irrigation.eq(1)
# irrigation_minor = crop_irrigation.eq(2)
# irrigation = irrigation_major.Or(irrigation_minor)


def process_crop(i, crop):
    for j, (geometry, name) in enumerate(regions_loop_2013):
#         print(j)
        for year in ['2016']:
            file_name = crop_names[i]+'_'+name.translate(translationTable)[:75]+'_'+year+'.pickle'
            if file_name not in files and file_name in just_file_names:
                try:
    
                    masked_soil_clim_land_image = ee.Image()

                    AAFC = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date(year+'-01-01', year+'-12-31')).first()
                    mask = AAFC.eq(crop)


                    # Previous Year Crops
                    for y in range(8):  # Loop through the previous 8 years
                        prev_year = int(year) - y
                        if prev_year >= 2013:
                            start_date = str(prev_year) + '-01-01'
                            end_date = str(prev_year) + '-12-31'
                            yearly_image = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date(start_date, end_date)).first()
                        else:
                            yearly_image = ee.Image.constant(0).rename('landcover')
                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([yearly_image])





                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([slope])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([elevation.mosaic()])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([aspect])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([soil_water_content])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([oc])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([pH])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([bulk])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([texture])
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([irrigation]) #irrigation status


                            
                    # Temporal Covariates
                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_prec = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
                                          .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                 ee.Date(year + '-' + month + '-' + str(week_end)))) \
                                          .select('precipitationCal') \
                                          .sum()  # Get the max precipitation for the week
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_prec.rename(f'precip_sum')])
                             
                        
                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for day in range(1, 28 + 1, 7): 
                            week_end = day + 6
                            week_temp_collection = ee.ImageCollection([])
                            for d in range(day, week_end + 1):
                                day_temp = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                            .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(d)).getRange('day')))

                                day_temp_max = day_temp.select('Tair_f_inst').max()
                                day_temp_min = day_temp.select('Tair_f_inst').min()
                                day_temp_diurnal = day_temp_max.subtract(day_temp_min)
                                week_temp_collection = week_temp_collection.merge(ee.ImageCollection([day_temp_diurnal]))

                            week_temp_max_diurnal = week_temp_collection.mean()
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_temp_max_diurnal.rename(f'diurnal_temp_mean')])

                            
                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_temp = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                          .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                 ee.Date(year + '-' + month + '-' + str(week_end)))) \
                                          .select('Tair_f_inst') \
                                          .max() 
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_temp.rename(f'Tair_f_inst_max')])


                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                            .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                   ee.Date(year + '-' + month + '-' + str(week_end))))
                            week_humidity = week_GLDAS.select('Qair_f_inst').mean()
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_humidity.rename(f'Qair_f_inst_mean')])


                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                            .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                   ee.Date(year + '-' + month + '-' + str(week_end))))
                            week_long_radiation = week_GLDAS.select('LWdown_f_tavg').sum()
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_long_radiation.rename(f'LWdown_f_tavg_sum')])
               

                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                            .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                   ee.Date(year + '-' + month + '-' + str(week_end))))
                            week_short_radiation = week_GLDAS.select('SWdown_f_tavg').sum()
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_short_radiation.rename(f'SWdown_f_tavg_sum')])

                            
                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_SMAP = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
                                          .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                 ee.Date(year + '-' + month + '-' + str(week_end))))

                            if week_SMAP.size().getInfo() != 0:
                                week_soil_moisture_top = week_SMAP.select('sm_surface').mean()
                            else:
                                week_soil_moisture_top = ee.Image.constant(0)
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_soil_moisture_top.rename(f'sm_surface_mean')])
                            
                            
                    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                        for week_start in range(1, 28 + 1, 7):
                            week_end = week_start + 6 
                            week_SMAP = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
                                          .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                 ee.Date(year + '-' + month + '-' + str(week_end))))

                            if week_SMAP.size().getInfo() != 0:
                                week_soil_moisture_root = week_SMAP.select('sm_rootzone').mean()
                            else:
                                week_soil_moisture_root = ee.Image.constant(0)
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_soil_moisture_root.rename(f'sm_rootzone_mean')])


                            
                            
                            
                    

                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands(masked_soil_clim_land_image.pixelArea())
                    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands(masked_soil_clim_land_image.pixelLonLat())
                    masked_soil_clim_land_image = masked_soil_clim_land_image.unmask(0).updateMask(mask)

                    try:
                        pixel_values = masked_soil_clim_land_image.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=500, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]])
                    except:
                        pixel_values = masked_soil_clim_land_image.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=1000, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]])
                        
                    if num_pixels > 40000:
                        pixel_values = masked_soil_clim_land_image.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=1000, maxPixels=1e13)
                        dictt = pixel_values.getInfo()
                        num_pixels = len(dictt[list(dictt.keys())[0]])

                    
                    if num_pixels > 0 and num_pixels <= 40000:

                        # Texture ----------------------------------------------------------------------------------------------------
                        data_labels = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]).reshape(-1, 1)
                        encoder = OneHotEncoder(sparse=False).fit(data_labels)

                        keys_with_texture = [key for key in dictt if 'texture' in key]
                        sorted_keys_texture = natsorted(keys_with_texture)
                        texture_v = np.array([dictt[k] for k in sorted_keys_texture]).transpose(1, 0)
                        non_zero_rows = ~np.all(texture_v == 0, axis=1)
                        texture_v = texture_v[non_zero_rows]
                        texture_v = encoder.transform(texture_v.reshape(-1, 1)).reshape(-1, 6, 12).transpose(0, 2, 1)
    #                     print('texture', texture_v.shape)


                        # Area, location --------------------------------------------------------------------------------------
                        area = np.array(dictt['area'])[non_zero_rows]
                        longitude = np.array(dictt['longitude'])[non_zero_rows]
                        latitude = np.array(dictt['latitude'])[non_zero_rows]
                        #print('area, lon, lat', area.shape, longitude.shape, latitude.shape)

                        # Time-Dependent -------------------------------------------------------------------------------------------------
                        keys_with_sm_surface = [key for key in dictt if 'sm_surface_mean' in key]
                        sorted_keys_sm_surface = natsorted(keys_with_sm_surface)
                        sm_surface_v = np.array([dictt[k] for k in sorted_keys_sm_surface]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_sm_rootzone = [key for key in dictt if 'sm_rootzone_mean' in key]
                        sorted_keys_sm_rootzone = natsorted(keys_with_sm_rootzone)
                        sm_rootzone_v = np.array([dictt[k] for k in sorted_keys_sm_rootzone]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_SWdown_f_tavg = [key for key in dictt if 'SWdown_f_tavg_sum' in key]
                        sorted_keys_SWdown_f_tavg = natsorted(keys_with_SWdown_f_tavg)
                        SWdown_f_tavg_v = np.array([dictt[k] for k in sorted_keys_SWdown_f_tavg]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_LWdown_f_tavg = [key for key in dictt if 'LWdown_f_tavg_sum' in key]
                        sorted_keys_LWdown_f_tavg = natsorted(keys_with_LWdown_f_tavg)
                        LWdown_f_tavg_v = np.array([dictt[k] for k in sorted_keys_LWdown_f_tavg]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_Qair_f_inst = [key for key in dictt if 'Qair_f_inst_mean' in key]
                        sorted_keys_Qair_f_inst = natsorted(keys_with_Qair_f_inst)
                        Qair_f_inst_v = np.array([dictt[k] for k in sorted_keys_Qair_f_inst]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_Tair_f_inst_max = [key for key in dictt if 'Tair_f_inst_max' in key]
                        sorted_keys_Tair_f_inst_max = natsorted(keys_with_Tair_f_inst_max)
                        Tair_f_inst_max_v = np.array([dictt[k] for k in sorted_keys_Tair_f_inst_max]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_Tair_f_inst = [key for key in dictt if 'diurnal_temp_mean' in key]
                        sorted_keys_Tair_f_inst = natsorted(keys_with_Tair_f_inst)
                        Tair_f_inst_v = np.array([dictt[k] for k in sorted_keys_Tair_f_inst]).transpose(1, 0).reshape(-1, 1, 48)

                        keys_with_precipitationCal = [key for key in dictt if 'precip_sum' in key]
                        sorted_keys_precipitationCal = natsorted(keys_with_precipitationCal)
                        precipitationCal_v = np.array([dictt[k] for k in sorted_keys_precipitationCal]).transpose(1, 0).reshape(-1, 1, 48)

                        time_dependent = np.concatenate([sm_surface_v, sm_rootzone_v, SWdown_f_tavg_v, LWdown_f_tavg_v, Qair_f_inst_v, Tair_f_inst_max_v, Tair_f_inst_v, precipitationCal_v], axis=1)
                        time_dependent = time_dependent[non_zero_rows]
    #                     print('time_dependent', time_dependent.shape)

                        # Soil -------------------------------------------------------------------------------------------------
                        keys_with_pH = [key for key in dictt if 'pH' in key]
                        sorted_keys_pH = natsorted(keys_with_pH)
                        pH_v = np.array([dictt[k] for k in sorted_keys_pH]).transpose(1, 0).reshape(-1, 1, 6)

                        keys_with_soil_water_content = [key for key in dictt if 'soil_water_content' in key]
                        sorted_keys_soil_water_content = natsorted(keys_with_soil_water_content)
                        soil_water_content_v = np.array([dictt[k] for k in sorted_keys_soil_water_content]).transpose(1, 0).reshape(-1, 1, 6)

                        keys_with_oc = [key for key in dictt if 'oc' in key]
                        sorted_keys_oc = natsorted(keys_with_oc)
                        oc_v = np.array([dictt[k] for k in sorted_keys_oc]).transpose(1, 0).reshape(-1, 1, 6)

                        keys_with_bulk = [key for key in dictt if 'bulk' in key]
                        sorted_keys_bulk = natsorted(keys_with_bulk)
                        bulk_v = np.array([dictt[k] for k in sorted_keys_bulk]).transpose(1, 0).reshape(-1, 1, 6)

                        soil = np.concatenate([pH_v, soil_water_content_v, oc_v, bulk_v], axis=1)
                        soil = soil[non_zero_rows]
                        #print('soil', soil.shape)

                        # Singles ------------------------------------------------------------------------------------------------------
                        keys_with_slope = [key for key in dictt if 'slope' in key]
                        sorted_keys_slope = natsorted(keys_with_slope)
                        slope_v = np.array([dictt[k] for k in sorted_keys_slope]).transpose(1, 0).reshape(-1, 1)

                        keys_with_aspect = [key for key in dictt if 'aspect' in key]
                        sorted_keys_aspect = natsorted(keys_with_aspect)
                        aspect_v = np.array([dictt[k] for k in sorted_keys_aspect]).transpose(1, 0).reshape(-1, 1)

                        keys_with_DSM = [key for key in dictt if 'DSM' in key]
                        sorted_keys_DSM = natsorted(keys_with_DSM)
                        DSM_v = np.array([dictt[k] for k in sorted_keys_DSM]).transpose(1, 0).reshape(-1, 1)


                        data_labels = np.array([0, 1, 2, 3, 4, 5]).reshape(-1, 1)
                        encoder = OneHotEncoder(sparse=False).fit(data_labels)
                        keys_with_irrigation = [key for key in dictt if 'irrigation' in key]
                        sorted_keys_irrigation = natsorted(keys_with_irrigation)
                        irrigation_v = np.array([dictt[k] for k in sorted_keys_irrigation]).transpose(1, 0)
                        irrigation_v = encoder.transform(irrigation_v.reshape(-1, 1)).reshape(-1, 6)

#                         singles = np.concatenate([slope_v, aspect_v, DSM_v, irrigation_v], axis=1)
                        singles = np.concatenate([slope_v, aspect_v, DSM_v], axis=1)
                        singles = singles[non_zero_rows]
                        #print('singles', singles.shape)

                        # Time series inventory-------------------------------------------------------------------------------------------
                        conversion_dict = {
                            0: 0, # mask
                            145: 24,  # Winter Wheat
                            174: 52,  # Lentils
                            147: 1,   # Corn
                            133: 21,  # Barley
                            142: 4,   # Sorghum
                            163: 51,  # Chickpeas
                            178: 41,  # Sugarbeets
                            155: 35,  # Mustard
                            136: 28,  # Oats
                            158: 5,   # Soy
                            153: 31,  # Canola
                            154: 32,  # Flaxseed
                            162: 53,  # Peas
                            146: 23,  # Spring Wheat
                            122: 176, # Grassland/Pasture
                        }
                        valid_values = {0, 1, 2, 3, 4, 5, 21, 22, 23, 24, 28, 31, 32,
                                        35, 36, 37, 41, 42, 43, 51, 52, 53, 141, 61, 176}
                        data_labels = np.array([0, 1, 2, 3, 4, 5, 21, 22, 23, 24, 28, 31, 32,
                                                35, 36, 37, 41, 42, 43, 51, 52, 53, 141, 61, 176, 999]).reshape(-1, 1)
                        encoder = OneHotEncoder(sparse=False).fit(data_labels)

                        keys_with_landcover = [key for key in dictt if 'landcover' in key]
                        sorted_keys_landcover = natsorted(keys_with_landcover)
                        landcover_v = np.array([dictt[k] for k in sorted_keys_landcover]).transpose(1, 0)
                        landcover_v = landcover_v[non_zero_rows]
                        landcover_v = np.array([conversion_dict.get(x, 999) if conversion_dict.get(x, 999) in valid_values else 999 for x in landcover_v.reshape(-1)]).reshape(-1, 8)
                        landcover_v = encoder.transform(landcover_v.reshape(-1, 1)).reshape(-1, 8, 26).transpose(0, 2, 1)

                        landcover_v = landcover_v[:, 1:, :]
                        #print('landcover', landcover_v.shape)



                        arrays = [area.astype(np.float32), longitude, latitude, time_dependent.astype(np.float32), soil.astype(np.int32), texture_v.astype(np.int32), singles.astype(np.float32), landcover_v.astype(np.int32)]
                        print(area.shape, longitude.shape, latitude.shape, time_dependent.shape, 
                              soil.shape, texture_v.shape, singles.shape, landcover_v.shape)
                        

                        with open('Raw_Data/TOSHIBA EXT/environment_ndvi_district/CA/'+file_name, 'wb') as file:
                            pickle.dump(arrays, file)
                            print(file_name)
                        
                except Exception as e:
                    print(f"An error occurred: {e}")
                    print(file_name)
                    pass




if __name__ == "__main__":
    
    # Create a multiprocessing pool
#     pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    pool = multiprocessing.Pool(processes=30)

    
    # Use pool.map to process each crop in parallel
    pool.starmap(process_crop, [(i, crop) for i, crop in enumerate(crops)])
    
    pool.close()
    pool.join()
              
            


    
##print('Done!')

#### USA

In [ ]:
import sys
import multiprocessing
from natsort import natsorted
from sklearn.preprocessing import OneHotEncoder
import time

new_limit = 15000
sys.setrecursionlimit(new_limit)

translationTable = str.maketrans("ÀàÇçÂâÄäÈèÉéÊêËÎîÏÔôÙùÛûÜüŸÿ()", "AaCcAaAaEeEeEeEIiIOoUuUuUuYy--")

while True:



    files = os.listdir('Raw_Data/TOSHIBA EXT/environment_ndvi_district/USA/')
    files = [x for x in files if x.endswith('.pickle')]


    crop_names = ['CORN, GRAIN - YIELD, MEASURED IN BU / ACRE',
                   'COTTON, UPLAND - YIELD, MEASURED IN LB / ACRE',
                   'PEANUTS - YIELD, MEASURED IN LB / ACRE',
                   'SOYBEANS - YIELD, MEASURED IN BU / ACRE',
                   'BARLEY - YIELD, MEASURED IN BU / ACRE',
                   'RICE - YIELD, MEASURED IN LB / ACRE',
                   'WHEAT, WINTER - YIELD, MEASURED IN BU / ACRE',
                   'OATS - YIELD, MEASURED IN BU / ACRE',
                   'SORGHUM, GRAIN - YIELD, MEASURED IN BU / ACRE',
                   'WHEAT, SPRING, (EXCL DURUM) - YIELD, MEASURED IN BU / ACRE',
                   'HAY, (EXCL ALFALFA) - YIELD, MEASURED IN TONS / ACRE',
                   'CANOLA - YIELD, MEASURED IN LB / ACRE',
                   'CHICKPEAS - YIELD, MEASURED IN LB / ACRE',
                   'LENTILS - YIELD, MEASURED IN LB / ACRE',
                   'PEAS, DRY EDIBLE - YIELD, MEASURED IN LB / ACRE',
                   'FLAXSEED - YIELD, MEASURED IN BU / ACRE',
                   'SUGARBEETS - YIELD, MEASURED IN TONS / ACRE',
    #               'SAFFLOWER - YIELD, MEASURED IN LB / ACRE', # not enough data
                   'MUSTARD - YIELD, MEASURED IN LB / ACRE']
    crops = [1, 2, 10, 5, 21, 3, 24, 28, 4, 23, 37, 31, 51, 52, 53, 32, 41, 35]
    
    
    
    crop_names = ['CHICKPEAS - YIELD, MEASURED IN LB / ACRE',
                   'MUSTARD - YIELD, MEASURED IN LB / ACRE']
    crops = [51, 35]



    elevation = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').select('DSM')
    proj_slope = elevation.first().select(0).projection()
    slope = ee.Terrain.slope(elevation.mosaic().setDefaultProjection(proj_slope))
    aspect = ee.Terrain.aspect(elevation.mosaic().setDefaultProjection(proj_slope))

    soil_water_content = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01")
    original_band_names = soil_water_content.bandNames()
    new_band_names = original_band_names.map(lambda name: ee.String('soil_water_content_').cat(name))
    soil_water_content = soil_water_content.select(original_band_names, new_band_names)

    oc = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02")
    original_band_names = oc.bandNames()
    new_band_names = original_band_names.map(lambda name: ee.String('oc_').cat(name))
    oc = oc.select(original_band_names, new_band_names)

    pH = ee.Image("OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02")
    original_band_names = pH.bandNames()
    new_band_names = original_band_names.map(lambda name: ee.String('pH_').cat(name))
    pH = pH.select(original_band_names, new_band_names)

    bulk = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02")
    original_band_names = bulk.bandNames()
    new_band_names = original_band_names.map(lambda name: ee.String('bulk_').cat(name))
    bulk = bulk.select(original_band_names, new_band_names)

    texture = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")
    original_band_names = texture.bandNames()
    new_band_names = original_band_names.map(lambda name: ee.String('texture_').cat(name))
    texture = texture.select(original_band_names, new_band_names)

    # Global map of lands that irrigate. This map is assumed to not change with time, which is wrong.
    irrigation = ee.Image('USGS/GFSAD1000_V1').select('landcover').rename('irrigation')
    # irrigation_major = crop_irrigation.eq(1)
    # irrigation_minor = crop_irrigation.eq(2)
    # irrigation = irrigation_major.Or(irrigation_minor)
    
    random.shuffle(usa_regions)

    def process_crop(i, crop):
        for j, (geometry, name) in enumerate(usa_regions):
    #         print(j)
            for year in ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']:
                file_name = crop_names[i].replace('/', '-').replace('(', '-').replace(')', '-')+'_'+name+'_'+year+'.pickle'
                if file_name not in files and file_name in just_file_names:
                    try:

                        masked_soil_clim_land_image = ee.Image()

                        # Mask for crop
                        AAFC = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(year+'-01-01', year+'-12-31')).first().select('cropland')
                        mask = AAFC.eq(crop)


                        # Previous Year Crop
                        for i in range(8):  # Loop through the previous 8 years
                            prev_year = int(year) - i
                            if prev_year >= 2012:
                                start_date = str(prev_year) + '-01-01'
                                end_date = str(prev_year) + '-12-31'
                                yearly_image = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(start_date, end_date)).first().select('cropland').rename('landcover')
                            else:
                                yearly_image = ee.Image.constant(0).rename('landcover')
                            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([yearly_image])



                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([slope])
                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([elevation.mosaic()])
                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([aspect])
                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([soil_water_content])
                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([oc])
                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([pH])
                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([bulk])
                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([texture])
                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([irrigation]) #irrigation status



                        # Temporal Covariates
                        for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                            for week_start in range(1, 28 + 1, 7):
                                week_end = week_start + 6 
                                week_prec = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
                                              .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                     ee.Date(year + '-' + month + '-' + str(week_end)))) \
                                              .select('precipitationCal') \
                                              .sum()  # Get the max precipitation for the week
                                masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_prec.rename(f'precip_sum')])


                        for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                            for day in range(1, 28 + 1, 7): 
                                week_end = day + 6
                                week_temp_collection = ee.ImageCollection([])
                                for d in range(day, week_end + 1):
                                    day_temp = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                                .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(d)).getRange('day')))

                                    day_temp_max = day_temp.select('Tair_f_inst').max()
                                    day_temp_min = day_temp.select('Tair_f_inst').min()
                                    day_temp_diurnal = day_temp_max.subtract(day_temp_min)
                                    week_temp_collection = week_temp_collection.merge(ee.ImageCollection([day_temp_diurnal]))

                                week_temp_max_diurnal = week_temp_collection.mean()
                                masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_temp_max_diurnal.rename(f'diurnal_temp_mean')])


                        for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                            for week_start in range(1, 28 + 1, 7):
                                week_end = week_start + 6 
                                week_temp = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                              .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                     ee.Date(year + '-' + month + '-' + str(week_end)))) \
                                              .select('Tair_f_inst') \
                                              .max() 
                                masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_temp.rename(f'Tair_f_inst_max')])


                        for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                            for week_start in range(1, 28 + 1, 7):
                                week_end = week_start + 6 
                                week_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                                .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                       ee.Date(year + '-' + month + '-' + str(week_end))))
                                week_humidity = week_GLDAS.select('Qair_f_inst').mean()
                                masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_humidity.rename(f'Qair_f_inst_mean')])


                        for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                            for week_start in range(1, 28 + 1, 7):
                                week_end = week_start + 6 
                                week_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                                .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                       ee.Date(year + '-' + month + '-' + str(week_end))))
                                week_long_radiation = week_GLDAS.select('LWdown_f_tavg').sum()
                                masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_long_radiation.rename(f'LWdown_f_tavg_sum')])


                        for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                            for week_start in range(1, 28 + 1, 7):
                                week_end = week_start + 6 
                                week_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
                                                .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                       ee.Date(year + '-' + month + '-' + str(week_end))))
                                week_short_radiation = week_GLDAS.select('SWdown_f_tavg').sum()
                                masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_short_radiation.rename(f'SWdown_f_tavg_sum')])


                        for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                            for week_start in range(1, 28 + 1, 7):
                                week_end = week_start + 6 
                                week_SMAP = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
                                              .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                     ee.Date(year + '-' + month + '-' + str(week_end))))

                                if week_SMAP.size().getInfo() != 0:
                                    week_soil_moisture_top = week_SMAP.select('sm_surface').mean()
                                else:
                                    week_soil_moisture_top = ee.Image.constant(0)
                                masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_soil_moisture_top.rename(f'sm_surface_mean')])


                        for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
                            for week_start in range(1, 28 + 1, 7):
                                week_end = week_start + 6 
                                week_SMAP = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
                                              .filter(ee.Filter.date(ee.Date(year + '-' + month + '-' + str(week_start)), 
                                                                     ee.Date(year + '-' + month + '-' + str(week_end))))

                                if week_SMAP.size().getInfo() != 0:
                                    week_soil_moisture_root = week_SMAP.select('sm_rootzone').mean()
                                else:
                                    week_soil_moisture_root = ee.Image.constant(0)
                                masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([week_soil_moisture_root.rename(f'sm_rootzone_mean')])







                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands(masked_soil_clim_land_image.pixelArea())
                        masked_soil_clim_land_image = masked_soil_clim_land_image.addBands(masked_soil_clim_land_image.pixelLonLat())
                        masked_soil_clim_land_image = masked_soil_clim_land_image.unmask(0).updateMask(mask)


                        try:
                            pixel_values = masked_soil_clim_land_image.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry.geometry(), scale=250, maxPixels=1e13)
                            dictt = pixel_values.getInfo()
                            num_pixels = len(dictt[list(dictt.keys())[0]])
                        except:
                            pixel_values = masked_soil_clim_land_image.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry.geometry(), scale=500, maxPixels=1e13)
                            dictt = pixel_values.getInfo()
                            num_pixels = len(dictt[list(dictt.keys())[0]])

                        if num_pixels > 40000:
                            pixel_values = masked_soil_clim_land_image.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry.geometry(), scale=500, maxPixels=1e13)
                            dictt = pixel_values.getInfo()
                            num_pixels = len(dictt[list(dictt.keys())[0]])


                        if num_pixels > 0 and num_pixels <= 40000:

                            # Texture ----------------------------------------------------------------------------------------------------
                            data_labels = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]).reshape(-1, 1)
                            encoder = OneHotEncoder(sparse=False).fit(data_labels)

                            keys_with_texture = [key for key in dictt if 'texture' in key]
                            sorted_keys_texture = natsorted(keys_with_texture)
                            texture_v = np.array([dictt[k] for k in sorted_keys_texture]).transpose(1, 0)
                            non_zero_rows = ~np.all(texture_v == 0, axis=1)
                            texture_v = texture_v[non_zero_rows]
                            texture_v = encoder.transform(texture_v.reshape(-1, 1)).reshape(-1, 6, 12).transpose(0, 2, 1)
        #                     print('texture', texture_v.shape)


                            # Area, location --------------------------------------------------------------------------------------
                            area = np.array(dictt['area'])[non_zero_rows]
                            longitude = np.array(dictt['longitude'])[non_zero_rows]
                            latitude = np.array(dictt['latitude'])[non_zero_rows]
                            #print('area, lon, lat', area.shape, longitude.shape, latitude.shape)

                            # Time-Dependent -------------------------------------------------------------------------------------------------
                            keys_with_sm_surface = [key for key in dictt if 'sm_surface_mean' in key]
                            sorted_keys_sm_surface = natsorted(keys_with_sm_surface)
                            sm_surface_v = np.array([dictt[k] for k in sorted_keys_sm_surface]).transpose(1, 0).reshape(-1, 1, 48)

                            keys_with_sm_rootzone = [key for key in dictt if 'sm_rootzone_mean' in key]
                            sorted_keys_sm_rootzone = natsorted(keys_with_sm_rootzone)
                            sm_rootzone_v = np.array([dictt[k] for k in sorted_keys_sm_rootzone]).transpose(1, 0).reshape(-1, 1, 48)

                            keys_with_SWdown_f_tavg = [key for key in dictt if 'SWdown_f_tavg_sum' in key]
                            sorted_keys_SWdown_f_tavg = natsorted(keys_with_SWdown_f_tavg)
                            SWdown_f_tavg_v = np.array([dictt[k] for k in sorted_keys_SWdown_f_tavg]).transpose(1, 0).reshape(-1, 1, 48)

                            keys_with_LWdown_f_tavg = [key for key in dictt if 'LWdown_f_tavg_sum' in key]
                            sorted_keys_LWdown_f_tavg = natsorted(keys_with_LWdown_f_tavg)
                            LWdown_f_tavg_v = np.array([dictt[k] for k in sorted_keys_LWdown_f_tavg]).transpose(1, 0).reshape(-1, 1, 48)

                            keys_with_Qair_f_inst = [key for key in dictt if 'Qair_f_inst_mean' in key]
                            sorted_keys_Qair_f_inst = natsorted(keys_with_Qair_f_inst)
                            Qair_f_inst_v = np.array([dictt[k] for k in sorted_keys_Qair_f_inst]).transpose(1, 0).reshape(-1, 1, 48)

                            keys_with_Tair_f_inst_max = [key for key in dictt if 'Tair_f_inst_max' in key]
                            sorted_keys_Tair_f_inst_max = natsorted(keys_with_Tair_f_inst_max)
                            Tair_f_inst_max_v = np.array([dictt[k] for k in sorted_keys_Tair_f_inst_max]).transpose(1, 0).reshape(-1, 1, 48)

                            keys_with_Tair_f_inst = [key for key in dictt if 'diurnal_temp_mean' in key]
                            sorted_keys_Tair_f_inst = natsorted(keys_with_Tair_f_inst)
                            Tair_f_inst_v = np.array([dictt[k] for k in sorted_keys_Tair_f_inst]).transpose(1, 0).reshape(-1, 1, 48)

                            keys_with_precipitationCal = [key for key in dictt if 'precip_sum' in key]
                            sorted_keys_precipitationCal = natsorted(keys_with_precipitationCal)
                            precipitationCal_v = np.array([dictt[k] for k in sorted_keys_precipitationCal]).transpose(1, 0).reshape(-1, 1, 48)

                            time_dependent = np.concatenate([sm_surface_v, sm_rootzone_v, SWdown_f_tavg_v, LWdown_f_tavg_v, Qair_f_inst_v, Tair_f_inst_max_v, Tair_f_inst_v, precipitationCal_v], axis=1)
                            time_dependent = time_dependent[non_zero_rows]
        #                     print('time_dependent', time_dependent.shape)

                            # Soil -------------------------------------------------------------------------------------------------
                            keys_with_pH = [key for key in dictt if 'pH' in key]
                            sorted_keys_pH = natsorted(keys_with_pH)
                            pH_v = np.array([dictt[k] for k in sorted_keys_pH]).transpose(1, 0).reshape(-1, 1, 6)

                            keys_with_soil_water_content = [key for key in dictt if 'soil_water_content' in key]
                            sorted_keys_soil_water_content = natsorted(keys_with_soil_water_content)
                            soil_water_content_v = np.array([dictt[k] for k in sorted_keys_soil_water_content]).transpose(1, 0).reshape(-1, 1, 6)

                            keys_with_oc = [key for key in dictt if 'oc' in key]
                            sorted_keys_oc = natsorted(keys_with_oc)
                            oc_v = np.array([dictt[k] for k in sorted_keys_oc]).transpose(1, 0).reshape(-1, 1, 6)

                            keys_with_bulk = [key for key in dictt if 'bulk' in key]
                            sorted_keys_bulk = natsorted(keys_with_bulk)
                            bulk_v = np.array([dictt[k] for k in sorted_keys_bulk]).transpose(1, 0).reshape(-1, 1, 6)

                            soil = np.concatenate([pH_v, soil_water_content_v, oc_v, bulk_v], axis=1)
                            soil = soil[non_zero_rows]
                            #print('soil', soil.shape)

                            # Singles ------------------------------------------------------------------------------------------------------
                            keys_with_slope = [key for key in dictt if 'slope' in key]
                            sorted_keys_slope = natsorted(keys_with_slope)
                            slope_v = np.array([dictt[k] for k in sorted_keys_slope]).transpose(1, 0).reshape(-1, 1)

                            keys_with_aspect = [key for key in dictt if 'aspect' in key]
                            sorted_keys_aspect = natsorted(keys_with_aspect)
                            aspect_v = np.array([dictt[k] for k in sorted_keys_aspect]).transpose(1, 0).reshape(-1, 1)

                            keys_with_DSM = [key for key in dictt if 'DSM' in key]
                            sorted_keys_DSM = natsorted(keys_with_DSM)
                            DSM_v = np.array([dictt[k] for k in sorted_keys_DSM]).transpose(1, 0).reshape(-1, 1)


                            data_labels = np.array([0, 1, 2, 3, 4, 5]).reshape(-1, 1)
                            encoder = OneHotEncoder(sparse=False).fit(data_labels)
                            keys_with_irrigation = [key for key in dictt if 'irrigation' in key]
                            sorted_keys_irrigation = natsorted(keys_with_irrigation)
                            irrigation_v = np.array([dictt[k] for k in sorted_keys_irrigation]).transpose(1, 0)
                            irrigation_v = encoder.transform(irrigation_v.reshape(-1, 1)).reshape(-1, 6)

    #                         singles = np.concatenate([slope_v, aspect_v, DSM_v, irrigation_v], axis=1)
                            singles = np.concatenate([slope_v, aspect_v, DSM_v], axis=1)
                            singles = singles[non_zero_rows]
                            #print('singles', singles.shape)

                            # Time series inventory-------------------------------------------------------------------------------------------
                            conversion_dict = {
                                0: 0, # mask
                                145: 24,  # Winter Wheat
                                174: 52,  # Lentils
                                147: 1,   # Corn
                                133: 21,  # Barley
                                142: 4,   # Sorghum
                                163: 51,  # Chickpeas
                                178: 41,  # Sugarbeets
                                155: 35,  # Mustard
                                136: 28,  # Oats
                                158: 5,   # Soy
                                153: 31,  # Canola
                                154: 32,  # Flaxseed
                                162: 53,  # Peas
                                146: 23,  # Spring Wheat
                                122: 176, # Grassland/Pasture
                            }
                            valid_values = {0, 1, 2, 3, 4, 5, 21, 22, 23, 24, 28, 31, 32,
                                            35, 36, 37, 41, 42, 43, 51, 52, 53, 141, 61, 176}
                            data_labels = np.array([0, 1, 2, 3, 4, 5, 21, 22, 23, 24, 28, 31, 32,
                                                    35, 36, 37, 41, 42, 43, 51, 52, 53, 141, 61, 176, 999]).reshape(-1, 1)
                            encoder = OneHotEncoder(sparse=False).fit(data_labels)

                            keys_with_landcover = [key for key in dictt if 'landcover' in key]
                            sorted_keys_landcover = natsorted(keys_with_landcover)
                            landcover_v = np.array([dictt[k] for k in sorted_keys_landcover]).transpose(1, 0)
                            landcover_v = landcover_v[non_zero_rows]
                            landcover_v = np.array([conversion_dict.get(x, 999) if conversion_dict.get(x, 999) in valid_values else 999 for x in landcover_v.reshape(-1)]).reshape(-1, 8)
                            landcover_v = encoder.transform(landcover_v.reshape(-1, 1)).reshape(-1, 8, 26).transpose(0, 2, 1)
                            landcover_v = landcover_v[:, 1:, :]
                            # print('landcover', landcover_v.shape)



                            arrays = [area.astype(np.float32), longitude, latitude, time_dependent.astype(np.float32), soil.astype(np.int32), texture_v.astype(np.int32), singles.astype(np.float32), landcover_v.astype(np.int32)]
                            print(area.shape, longitude.shape, latitude.shape, time_dependent.shape, 
                                  soil.shape, texture_v.shape, singles.shape, landcover_v.shape)


                            with open('Raw_Data/TOSHIBA EXT/environment_ndvi_district/USA/'+file_name, 'wb') as file:
                                pickle.dump(arrays, file)
                                print(file_name)

                    except Exception as e:
                        print(f"An error occurred: {e}")
                        print(file_name)
                        pass




    if __name__ == "__main__":

        # Create a multiprocessing pool
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
#         pool = multiprocessing.Pool(processes=30)


        # Use pool.map to process each crop in parallel
        pool.starmap(process_crop, [(i, crop) for i, crop in enumerate(crops)])

        pool.close()
        pool.join()

            


    
##print('Done!')

### Download Projections for Canada 2020-2023

In [33]:
import sys
import multiprocessing
from natsort import natsorted
from sklearn.preprocessing import OneHotEncoder

new_limit = 15000
sys.setrecursionlimit(new_limit)

Canada_boundary = ee.FeatureCollection('users/amanjotbhullar/lcd_000a16a_e').geometry().dissolve()

elevation = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').select('DSM')
proj_slope = elevation.first().select(0).projection()
slope = ee.Terrain.slope(elevation.mosaic().setDefaultProjection(proj_slope))
aspect = ee.Terrain.aspect(elevation.mosaic().setDefaultProjection(proj_slope))

soil_water_content = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01")
original_band_names = soil_water_content.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('soil_water_content_').cat(name))
soil_water_content = soil_water_content.select(original_band_names, new_band_names)

oc = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02")
original_band_names = oc.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('oc_').cat(name))
oc = oc.select(original_band_names, new_band_names)

pH = ee.Image("OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02")
original_band_names = pH.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('pH_').cat(name))
pH = pH.select(original_band_names, new_band_names)

bulk = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02")
original_band_names = bulk.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('bulk_').cat(name))
bulk = bulk.select(original_band_names, new_band_names)

texture = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")
original_band_names = texture.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('texture_').cat(name))
texture = texture.select(original_band_names, new_band_names)



for year in range(2020, 2024):
    year = str(year)
    masked_soil_clim_land_image = ee.Image()


    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([slope])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([elevation.mosaic()])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([aspect])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([soil_water_content])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([oc])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([pH])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([bulk])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([texture])


    # Temporal Covariates
    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_prec = ee.ImageCollection('NASA/GPM_L3/IMERG_V06').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day')))
            day_prec_mean = day_prec.select('precipitationCal').mean()
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_prec_mean])



    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_temp = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day')))
            day_temp_max = day_temp.select('Tair_f_inst').max()
            day_temp_min = day_temp.select('Tair_f_inst').min()
            day_temp_diurnal = day_temp_max.subtract(day_temp_min)
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_temp_diurnal])


    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_prec = ee.ImageCollection('NASA/GPM_L3/IMERG_V06').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day')))
            day_prec_std = day_prec.select('precipitationCal').reduce(ee.Reducer.stdDev()).rename('precipitationCal_std')
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_prec_std])



    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_temp = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day')))
            day_temp_max = day_temp.select('Tair_f_inst').max().rename('Tair_f_inst_max')
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_temp_max])


    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day')))
            day_humidity = day_GLDAS.select('Qair_f_inst').mean()
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_humidity])


    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day')))
            day_long_radiation = day_GLDAS.select('LWdown_f_tavg').mean()
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_long_radiation])



    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day')))
            day_short_radiation = day_GLDAS.select('SWdown_f_tavg').mean()
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_short_radiation])




    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands(masked_soil_clim_land_image.pixelArea())
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands(masked_soil_clim_land_image.pixelLonLat())
    
    geemap.ee_export_image_to_drive(masked_soil_clim_land_image.toFloat(), 
                                            description=year, 
                                            folder='export', region=Canada_boundary, scale=2000)






### Download Projections for Canada 2050, 2100

Note: You will need to manually subtract to get diurnal when data is downloaded.

In [32]:
import sys
import multiprocessing
from natsort import natsorted
from sklearn.preprocessing import OneHotEncoder

new_limit = 15000
sys.setrecursionlimit(new_limit)

Canada_boundary = ee.FeatureCollection('users/amanjotbhullar/lcd_000a16a_e').geometry().dissolve()

elevation = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').select('DSM')
proj_slope = elevation.first().select(0).projection()
slope = ee.Terrain.slope(elevation.mosaic().setDefaultProjection(proj_slope))
aspect = ee.Terrain.aspect(elevation.mosaic().setDefaultProjection(proj_slope))

soil_water_content = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01")
original_band_names = soil_water_content.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('soil_water_content_').cat(name))
soil_water_content = soil_water_content.select(original_band_names, new_band_names)

oc = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02")
original_band_names = oc.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('oc_').cat(name))
oc = oc.select(original_band_names, new_band_names)

pH = ee.Image("OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02")
original_band_names = pH.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('pH_').cat(name))
pH = pH.select(original_band_names, new_band_names)

bulk = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02")
original_band_names = bulk.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('bulk_').cat(name))
bulk = bulk.select(original_band_names, new_band_names)

texture = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")
original_band_names = texture.bandNames()
new_band_names = original_band_names.map(lambda name: ee.String('texture_').cat(name))
texture = texture.select(original_band_names, new_band_names)


for year, scenario in [(2050, 'ssp245'), (2100, 'ssp245'), (2050, 'ssp585'), (2100, 'ssp585')]:
    year = str(year)
    masked_soil_clim_land_image = ee.Image()


    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([slope])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([elevation.mosaic()])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([aspect])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([soil_water_content])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([oc])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([pH])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([bulk])
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([texture])


    # Temporal Covariates
    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_prec = ee.ImageCollection('NASA/GDDP-CMIP6').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day'))).filter(ee.Filter.eq('model', 'CanESM5')).filter(ee.Filter.eq('scenario', scenario))
            day_prec_mean = day_prec.select('pr').first().multiply(3600)
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_prec_mean])



    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_temp = ee.ImageCollection('NASA/GDDP-CMIP6').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day'))).filter(ee.Filter.eq('model', 'CanESM5')).filter(ee.Filter.eq('scenario', scenario))
            day_temp_min = day_temp.select('tasmin').first()
            day_temp_max = day_temp.select('tasmax').first()
            day_temp_diurnal = day_temp_max.subtract(day_temp_min)
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_temp_diurnal])


    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_temp = ee.ImageCollection('NASA/GDDP-CMIP6').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day'))).filter(ee.Filter.eq('model', 'CanESM5')).filter(ee.Filter.eq('scenario', scenario))
            day_temp_max = day_temp.select('tasmax').first().rename('tasmax_max')
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_temp_max])


    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_GLDAS = ee.ImageCollection('NASA/GDDP-CMIP6').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day'))).filter(ee.Filter.eq('model', 'CanESM5')).filter(ee.Filter.eq('scenario', scenario))
            day_humidity = day_GLDAS.select('huss').first()
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_humidity])


    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_GLDAS = ee.ImageCollection('NASA/GDDP-CMIP6').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day'))).filter(ee.Filter.eq('model', 'CanESM5')).filter(ee.Filter.eq('scenario', scenario))
            day_long_radiation = day_GLDAS.select('rlds').first()
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_long_radiation])



    for month, days in [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'), ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]:
        for day in range(1, int(days)+1):
            day_GLDAS = ee.ImageCollection('NASA/GDDP-CMIP6').filter(ee.Filter.date(ee.Date(year+'-'+month+'-'+str(day)).getRange('day'))).filter(ee.Filter.eq('model', 'CanESM5')).filter(ee.Filter.eq('scenario', scenario))
            day_short_radiation = day_GLDAS.select('rsds').first()
            masked_soil_clim_land_image = masked_soil_clim_land_image.addBands([day_short_radiation])




    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands(masked_soil_clim_land_image.pixelArea())
    masked_soil_clim_land_image = masked_soil_clim_land_image.addBands(masked_soil_clim_land_image.pixelLonLat())
    
    geemap.ee_export_image_to_drive(masked_soil_clim_land_image.toFloat(), 
                                            description=year+'_'+scenario, 
                                            folder='export', region=Canada_boundary, scale=2000)






## Crop Rotation

Crop Rotation Table Canada

In [ ]:
from collections import OrderedDict
import seaborn as sns

crop_mapping = {
    10: 'Cloud',
    20: 'Water',
    30: 'Exposed Land and Barren',
    34: 'Urban and Developed',
    35: 'Greenhouses',
    50: 'Shrubland',
    80: 'Wetland',
    85: 'Peatland',
    110: 'Grassland',
    120: 'Agriculture (undifferentiated)',
    122: 'Pasture and Forages',
    130: 'Too Wet to be Seeded',
    131: 'Fallow',
    132: 'Cereals',
    133: 'Barley',
    134: 'Other Grains',
    135: 'Millet',
    136: 'Oats',
    137: 'Rye',
    138: 'Spelt',
    139: 'Triticale',
    140: 'Wheat',
    141: 'Switchgrass',
    142: 'Sorghum',
    143: 'Quinoa',
    145: 'Winter Wheat',
    146: 'Spring Wheat',
    147: 'Corn',
    148: 'Tobacco',
    149: 'Ginseng',
    150: 'Oilseeds',
    151: 'Borage',
    152: 'Camelina',
    153: 'Canola and Rapeseed',
    154: 'Flaxseed',
    155: 'Mustard',
    156: 'Safflower',
    157: 'Sunflower',
    158: 'Soybeans',
    160: 'Pulses',
    161: 'Other Pulses',
    162: 'Peas',
    163: 'Chickpeas',
    167: 'Beans',
    168: 'Fababeans',
    174: 'Lentils',
    175: 'Vegetables',
    176: 'Tomatoes',
    177: 'Potatoes',
    178: 'Sugarbeets',
    179: 'Other Vegetables',
    180: 'Fruits',
    181: 'Berries',
    182: 'Blueberry',
    183: 'Cranberry',
    185: 'Other Berry',
    188: 'Orchards',
    189: 'Other Fruits',
    190: 'Vineyards',
    191: 'Hops',
    192: 'Sod',
    193: 'Herbs',
    194: 'Nursery',
    195: 'Buckwheat',
    196: 'Canaryseed',
    197: 'Hemp',
    198: 'Vetch',
    199: 'Other Crops',
    200: 'Forest (undifferentiated)',
    210: 'Coniferous',
    220: 'Broadleaf',
    230: 'Mixedwood'
}



crop_names = ['Barley', 'Oats', 'Soybeans', 'Canola', 'Flaxseed', 'Peas, dry', 'Wheat, spring']
crops = [133, 136, 158, 153, 154, 162, 146]


unique_keys = []
dict1s = []
for i, crop in enumerate(crops):
    dict1 = {}
    for year in range(2013, 2020):

        AAFC = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date(str(year+1)+'-01-01', str(year+1)+'-12-31')).first()
        mask = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date(str(year)+'-01-01', str(year)+'-12-31')).first().eq(crop)
        image = AAFC.updateMask(mask)
        
        unique_values = image.select('landcover').reduceRegion(reducer=ee.Reducer.frequencyHistogram(),
                                                                geometry=image.geometry(),
                                                                scale=500, maxPixels=1e13)
        dict2 = unique_values.getInfo()['landcover']
        dict1 = {key: dict1.get(key, 0) + dict2.get(key, 0) for key in set(dict1) | set(dict2)}
        
    # Normalize and make it look nice    
    dict1 = {crop_mapping[int(key)]: value for key, value in dict1.items()}
    total_sum = sum(dict1.values())
    dict1 = {key: round((value / total_sum)*100, 2) for key, value in dict1.items()}
    dict1s.append(dict1)
    unique_keys = unique_keys + list(dict1)
    
unique_keys = list(set(unique_keys))

for dict1 in dict1s:
    for unique_key in unique_keys:
        dict1.setdefault(unique_key, 0)
dict1s = [OrderedDict(sorted(dict1.items())) for dict1 in dict1s]

#------------------------ Plot-------------------------------------------------

crop_rotation = np.array([list(dict1.values()) for dict1 in dict1s])
x_labels = list(dict1s[0])
y_labels = crop_names

# Find columns where all elements are less than 0.05
columns_to_remove = np.all(crop_rotation < 5, axis=0)
crop_rotation = crop_rotation[:, ~columns_to_remove]
x_labels = [item for i, item in enumerate(x_labels) if columns_to_remove[i]==False]


plt.figure(figsize=(14, 6))
sns.heatmap(crop_rotation,  xticklabels=x_labels, yticklabels=y_labels, cmap='Blues', annot=True, fmt='.1f', cbar=False) 
plt.show()

Crop Rotation Table USA

In [ ]:
from collections import OrderedDict
import seaborn as sns

crop_mapping = {
    1: 'Corn',
    2: 'Cotton',
    3: 'Rice',
    4: 'Sorghum',
    5: 'Soybeans',
    6: 'Sunflower',
    10: 'Peanuts',
    11: 'Tobacco',
    12: 'Sweet Corn',
    13: 'Pop or Orn Corn',
    14: 'Mint',
    21: 'Barley',
    22: 'Durum Wheat',
    23: 'Spring Wheat',
    24: 'Winter Wheat',
    25: 'Other Small Grains',
    26: 'Dbl Crop WinWht/Soybeans',
    27: 'Rye',
    28: 'Oats',
    29: 'Millet',
    30: 'Speltz',
    31: 'Canola',
    32: 'Flaxseed',
    33: 'Safflower',
    34: 'Rape Seed',
    35: 'Mustard',
    36: 'Alfalfa',
    37: 'Other Hay/Non Alfalfa',
    38: 'Camelina',
    39: 'Buckwheat',
    41: 'Sugarbeets',
    42: 'Dry Beans',
    43: 'Potatoes',
    44: 'Other Crops',
    45: 'Sugarcane',
    46: 'Sweet Potatoes',
    47: 'Misc Vegs & Fruits',
    48: 'Watermelons',
    49: 'Onions',
    50: 'Cucumbers',
    51: 'Chick Peas',
    52: 'Lentils',
    53: 'Peas',
    54: 'Tomatoes',
    55: 'Caneberries',
    56: 'Hops',
    57: 'Herbs',
    58: 'Clover/Wildflowers',
    59: 'Sod/Grass Seed',
    60: 'Switchgrass',
    61: 'Fallow/Idle Cropland',
    63: 'Forest',
    64: 'Shrubland',
    65: 'Barren',
    66: 'Cherries',
    67: 'Peaches',
    68: 'Apples',
    69: 'Grapes',
    70: 'Christmas Trees',
    71: 'Other Tree Crops',
    72: 'Citrus',
    74: 'Pecans',
    75: 'Almonds',
    76: 'Walnuts',
    77: 'Pears',
    81: 'Clouds/No Data',
    82: 'Developed',
    83: 'Water',
    87: 'Wetlands',
    88: 'Nonag/Undefined',
    92: 'Aquaculture',
    111: 'Open Water',
    112: 'Perennial Ice/Snow',
    121: 'Developed/Open Space',
    122: 'Developed/Low Intensity',
    123: 'Developed/Med Intensity',
    124: 'Developed/High Intensity',
    131: 'Barren',
    141: 'Deciduous Forest',
    142: 'Evergreen Forest',
    143: 'Mixed Forest',
    152: 'Shrubland',
    176: 'Grassland/Pasture',
    190: 'Woody Wetlands',
    195: 'Herbaceous Wetlands',
    204: 'Pistachios',
    205: 'Triticale',
    206: 'Carrots',
    207: 'Asparagus',
    208: 'Garlic',
    209: 'Cantaloupes',
    210: 'Prunes',
    211: 'Olives',
    212: 'Oranges',
    213: 'Honeydew Melons',
    214: 'Broccoli',
    215: 'Avocados',
    216: 'Peppers',
    217: 'Pomegranates',
    218: 'Nectarines',
    219: 'Greens',
    220: 'Plums',
    221: 'Strawberries',
    222: 'Squash',
    223: 'Apricots',
    224: 'Vetch',
    225: 'Dbl Crop WinWht/Corn',
    226: 'Dbl Crop Oats/Corn',
    227: 'Lettuce',
    228: 'Dbl Crop Triticale/Corn',
    229: 'Pumpkins',
    230: 'Dbl Crop Lettuce/Durum Wht',
    231: 'Dbl Crop Lettuce/Cantaloupe',
    232: 'Dbl Crop Lettuce/Cotton',
    233: 'Dbl Crop Lettuce/Barley',
    234: 'Dbl Crop Durum Wht/Sorghum',
    235: 'Dbl Crop Barley/Sorghum',
    236: 'Dbl Crop WinWht/Sorghum',
    237: 'Dbl Crop Barley/Corn',
    238: 'Dbl Crop WinWht/Cotton',
    239: 'Dbl Crop Soybeans/Cotton',
    240: 'Dbl Crop Soybeans/Oats',
    241: 'Dbl Crop Corn/Soybeans',
    242: 'Blueberries',
    243: 'Cabbage',
    244: 'Cauliflower',
    245: 'Celery',
    246: 'Radishes',
    247: 'Turnips',
    248: 'Eggplants',
    249: 'Gourds',
    250: 'Cranberries',
    254: 'Dbl Crop Barley/Soybeans'
}




crop_names = ['BARLEY - YIELD, MEASURED IN BU / ACRE',
                 'CANOLA - YIELD, MEASURED IN LB / ACRE',
                 'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE',
                 'CORN, GRAIN, IRRIGATED - YIELD, MEASURED IN BU / ACRE',
                 'CORN, GRAIN, NON-IRRIGATED - YIELD, MEASURED IN BU / ACRE',
                 'COTTON, UPLAND - YIELD, MEASURED IN LB / ACRE',
                 'COTTON, UPLAND, IRRIGATED - YIELD, MEASURED IN LB / ACRE',
                 'COTTON, UPLAND, NON-IRRIGATED - YIELD, MEASURED IN LB / ACRE',
                 'FLAXSEED - YIELD, MEASURED IN BU / ACRE',
                 'HAY, (EXCL ALFALFA) - YIELD, MEASURED IN TONS / ACRE',
                 'LENTILS - YIELD, MEASURED IN LB / ACRE',
                 'OATS - YIELD, MEASURED IN BU / ACRE',
                 'PEAS, DRY EDIBLE - YIELD, MEASURED IN LB / ACRE',
                 'POTATOES - YIELD, MEASURED IN CWT / ACRE',
                 'RICE - YIELD, MEASURED IN LB / ACRE',
                 'SORGHUM, GRAIN - YIELD, MEASURED IN BU / ACRE',
                 'SORGHUM, GRAIN, IRRIGATED - YIELD, MEASURED IN BU / ACRE',
                 'SORGHUM, GRAIN, NON-IRRIGATED - YIELD, MEASURED IN BU / ACRE',
                 'SOYBEANS - YIELD, MEASURED IN BU / ACRE',
                 'SOYBEANS, IRRIGATED - YIELD, MEASURED IN BU / ACRE',
                 'SOYBEANS, NON-IRRIGATED - YIELD, MEASURED IN BU / ACRE',
                 'SUGARBEETS - YIELD, MEASURED IN TONS / ACRE',
                 'WHEAT, SPRING, (EXCL DURUM) - YIELD, MEASURED IN BU / ACRE',
                 'WHEAT, SPRING, (EXCL DURUM), IRRIGATED - YIELD, MEASURED IN BU / ACRE',
                 'WHEAT, SPRING, (EXCL DURUM), NON-IRRIGATED - YIELD, MEASURED IN BU / ACRE',
                 'WHEAT, SPRING, DURUM - YIELD, MEASURED IN BU / ACRE',
                 'WHEAT, WINTER - YIELD, MEASURED IN BU / ACRE',
                 'WHEAT, WINTER, IRRIGATED - YIELD, MEASURED IN BU / ACRE',
                 'WHEAT, WINTER, NON-IRRIGATED - YIELD, MEASURED IN BU / ACRE']
crops = [21, 31, 1, 1, 1, 2, 2, 2, 32, 37, 52, 
         28, 53, 43, 3, 4, 4, 4, 5, 5, 5, 41,
         23, 23, 23, 22, 24, 24, 24]


unique_keys = []
dict1s = []
for i, crop in enumerate(crops):
    dict1 = {}
    for year in range(2012, 2019):

        AAFC = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(str(year+1)+'-01-01', str(year+1)+'-12-31')).first().select('cropland')
        mask = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(str(year)+'-01-01', str(year)+'-12-31')).first().select('cropland').eq(crop)
        image = AAFC.updateMask(mask)
        
        unique_values = image.select('cropland').reduceRegion(reducer=ee.Reducer.frequencyHistogram(),
                                                                geometry=image.geometry(),
                                                                scale=500, maxPixels=1e13)
        dict2 = unique_values.getInfo()['cropland']
        dict1 = {key: dict1.get(key, 0) + dict2.get(key, 0) for key in set(dict1) | set(dict2)}
        
    # Normalize and make it look nice    
    dict1 = {crop_mapping[int(key)]: value for key, value in dict1.items()}
    total_sum = sum(dict1.values())
    dict1 = {key: round((value / total_sum)*100, 2) for key, value in dict1.items()}
    dict1s.append(dict1)
    unique_keys = unique_keys + list(dict1)
    
unique_keys = list(set(unique_keys))

for dict1 in dict1s:
    for unique_key in unique_keys:
        dict1.setdefault(unique_key, 0)
dict1s = [OrderedDict(sorted(dict1.items())) for dict1 in dict1s]

#------------------------ Plot-------------------------------------------------

crop_rotation = np.array([list(dict1.values()) for dict1 in dict1s])
x_labels = list(dict1s[0])
y_labels = [crop_name.split('-')[0] for crop_name in crop_names]

# Find columns where all elements are less than 0.05
columns_to_remove = np.all(crop_rotation < 5, axis=0)
crop_rotation = crop_rotation[:, ~columns_to_remove]
x_labels = [item for i, item in enumerate(x_labels) if columns_to_remove[i]==False]


plt.figure(figsize=(14, 6))
sns.heatmap(crop_rotation,  xticklabels=x_labels, yticklabels=y_labels, cmap='Blues', annot=True, fmt='.1f', cbar=False) 
plt.show()